In [201]:
import pandas as pd 
import numpy as np
import datetime
import importlib
from datetime import datetime,timedelta,date
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
from pulp import LpVariable, LpProblem, LpMinimize, lpSum, LpStatus ,LpMaximize,PULP_CBC_CMD

import space_opt_util as space_util
importlib.reload(space_util)

<module 'space_opt_util' from 'C:\\Users\\40107566\\OneDrive - Anheuser-Busch InBev\\Modeling\\space_opt_util.py'>

In [202]:
fecha = '14-09-2023'
fecha_dt = datetime.strptime(fecha, '%d-%m-%Y')
##############
#fecha_dt = date.today()
####################

fecha_str=datetime.strftime(fecha_dt, '%d-%m-%Y')
fecha_new_str= datetime.strftime(fecha_dt, "%d.%m.%Y")
day = fecha_dt.strftime("%A") 

#### what is this block saying?? what if the day is saturday? - Sejuti
### each day is named here
if day=='Sunday':
    day_of_week=day    
elif day!='Saturday':
    day_of_week="Weekday"
else:
    day_of_week=day

In [203]:
default_start_time ='00:00:01'

In [204]:
fecha_str

'14-09-2023'

In [205]:
day

'Thursday'

### SKU Attribute file

In [206]:
sku_description=pd.read_excel('Data/HO Master Data Materials FG 2023.xlsx')

In [207]:
sku_description.head(n=2)

,Sap Code,Description,Size,Pack,Category,Brand,Box x Hls,Box x Pallet,Weight x Box,Units x Box,Active Code,Weight x Pallet,HLS x Box
0,H1004,BARENA EXPORTACION 12OZ 24UD,12 ONZ,VIDRIO NR,CERVEZA,BARENA,11.741,49.0,39.06,24,No,1981.94,0.085172
1,H1005,PORT ROYAL EXPORTACION 12OZ,12 ONZ,VIDRIO NR,CERVEZA,PORT ROYAL,11.741,49.0,39.06,24,No,1981.94,0.085172


In [208]:
def read_sku_description(sku_description):
    #reads sku description 
    sku_description.columns = sku_description.columns.str.strip()
    #print(sku_description.columns)
    #remove rows where code is null and hl per pallet is null
    sku_description=sku_description[~sku_description['Sap Code'].isnull()]
    sku_description=sku_description[~sku_description['Weight x Pallet'].isnull()]
    sku_description=sku_description[~sku_description['Box x Pallet'].isnull()]
    sku_description=sku_description.drop_duplicates(subset=['Sap Code'],keep='first')
    sku_description=sku_description.rename(columns={'Sap Code': 'codigo','Weight x Pallet':'pound_weight_per_pallet','Box x Pallet':'package_per_pallet'})
    sku_description['kg_weight_per_pallet']=sku_description['pound_weight_per_pallet']*0.453592
    return sku_description

In [209]:
sku_description=read_sku_description(sku_description)

In [210]:
sku_description.head(n=2)

,codigo,Description,Size,Pack,Category,Brand,Box x Hls,package_per_pallet,Weight x Box,Units x Box,Active Code,pound_weight_per_pallet,HLS x Box,kg_weight_per_pallet
0,H1004,BARENA EXPORTACION 12OZ 24UD,12 ONZ,VIDRIO NR,CERVEZA,BARENA,11.741,49.0,39.06,24,No,1981.94,0.085172,898.992128
1,H1005,PORT ROYAL EXPORTACION 12OZ,12 ONZ,VIDRIO NR,CERVEZA,PORT ROYAL,11.741,49.0,39.06,24,No,1981.94,0.085172,898.992128


### Fleet data- Attribute and Currently availability and Ficha level Restriction

Fleet Attribute

In [211]:
def get_static_fleet_attribute_data(df_fleet):
    #Read and preprocess static fleet attribute data 
    df_fleet.columns = df_fleet.columns.str.strip()
    df_fleet=df_fleet.rename(columns={'Transporte2': 'Tipo de Vehículo','Empresa':'Modelo','Remolque2':'Ficha','Pallets':'Capacidad Pallets'})
    df_fleet['Capacidad Ton']=np.where(df_fleet['Tipo de Vehículo']=='Terceros',37,41)
    df_fleet['Capacidad KG_max'] =df_fleet['Capacidad Ton']*1000
    #df_fleet['Capacidad Ton'] =df_fleet['Peso Lbs']*0.00110231
    df_fleet_list=df_fleet[~df_fleet['Tipo de Vehículo'].isnull()]
    df_fleet_list['BREWERY']='all'
    #df_fleet_list['Capacidad Ton']= np.where(df_fleet_list['Capacidad Pallets']==24.0, 40.0, df_fleet_list['Capacidad Ton'])
    df_fleet_list['Horas de utilidad'] = '24 Hr'
    df_fleet_list=df_fleet_list[['Ficha', 'Tipo de Vehículo', 'Modelo','Capacidad KG_max','Capacidad Ton', 'Capacidad Pallets','Horas de utilidad','BREWERY']]
    #df_fleet_list=df_fleet_list[~(df_fleet_list.Ficha.isin(ficha_do_not_use_list)) ]
    return df_fleet_list

In [212]:
df_fleet=pd.read_excel(r'Data/Fleet.xlsx',sheet_name='Remolque')

In [213]:
df_fleet_list=get_static_fleet_attribute_data(df_fleet)

In [214]:
df_fleet_list.head(n=2)

,Ficha,Tipo de Vehículo,Modelo,Capacidad KG_max,Capacidad Ton,Capacidad Pallets,Horas de utilidad,BREWERY
0,HN-MR88,Hibrida,NUÑEZ,41000,41,28,24 Hr,all
1,HN-MR93,Hibrida,EMTRATECA,41000,41,28,24 Hr,all


In [215]:
df_fleet_list.shape

(258, 8)

Current Fleet data

In [216]:
def format2(str1):
    ##convert time from 1900-01-01 02:00:00	 to 02:00:00
    result=str1
    if len(str1.split())==2:
        result=str1.split()[1]
    return result 

In [217]:
def format1(str1):
    ##convert time from 1900-01-01 02:00:00	 to 02:00:00
    result=str1
    if len(str1.split())==2:
        result=str1.split()[0]
    return result 

In [218]:
def assign_initial_time(current_day_fleet_siders,fecha_next_str,default_start_time,day):
    import datetime
    from datetime import timedelta
    Fleet_start_date_time_org=pd.to_datetime(fecha_next_str+" "+default_start_time, format='%d-%m-%Y %H:%M:%S')
    if day=='Monday':
        Max_reach_time_at_destiny=Fleet_start_date_time_org+datetime.timedelta(hours=17)
    else:
        Max_reach_time_at_destiny=Fleet_start_date_time_org+datetime.timedelta(hours=23)
    current_day_fleet_siders['Fleet_start_time']=default_start_time
    current_day_fleet_siders['Fleet_start_date_time']=pd.to_datetime(fecha_next_str+" "+default_start_time, format='%d-%m-%Y %H:%M:%S')
    current_day_fleet_siders['Fleet_start_date_time_org']=Fleet_start_date_time_org
    current_day_fleet_siders['Max_reach_time_at_destiny']=Max_reach_time_at_destiny
    #print(type(Fleet_start_date_time_org))
    #print(type(Max_reach_time_at_destiny))
    current_day_fleet_siders['Fleet_start_time']=pd.to_datetime(current_day_fleet_siders['Fleet_start_time'], format='%H:%M:%S').dt.time
    return current_day_fleet_siders

In [219]:
def get_current_date_available_sider(df_fleet_list,Fleet_availability,Fleet_weight_tractor,Fleet_weight_remolque,default_start_time,fecha_next_str,day): 
    Fleet_availability=Fleet_availability[['Combinations Final']]
    Fleet_availability=Fleet_availability.dropna(subset = ['Combinations Final'])
    Fleet_availability['Remolque'] =Fleet_availability['Combinations Final'].astype(str).apply(format2)
    Fleet_availability['Tractor'] =Fleet_availability['Combinations Final'].astype(str).apply(format1)
    Fleet_availability=Fleet_availability.merge(Fleet_weight_tractor,left_on='Tractor',right_on='Tractor ID ',how='left').merge(Fleet_weight_remolque[['Remolque_ID', 'Weight_of_Remolque_in_KG']],left_on='Remolque',right_on='Remolque_ID',how='left')
    Fleet_availability=Fleet_availability.merge(df_fleet_list, left_on='Remolque',right_on='Ficha',how='left')
    Fleet_availability['Capacidad KG']=Fleet_availability['Capacidad KG_max']-(Fleet_availability['Weight of tractor in KG ']+Fleet_availability['Weight_of_Remolque_in_KG'])
    Fleet_availability = Fleet_availability.dropna(how='any',axis=0) 
    #current_day_fleet=current_day_fleet[~(current_day_fleet.Ficha.isin(ficha_do_not_use_list)) ]
    #current_day_fleet=df_fleet_list[df_fleet_list['Ficha'].isin(Current_fleet_list)]
    #lets add code to the brewery name 
#     dict1={'CND':'DO31','HN':'HN01'}
#     current_day_fleet['BREWERY_CODE']=current_day_fleet['BREWERY'].map(dict1)
    current_day_fleet_siders=Fleet_availability.copy(deep=True)
    current_day_fleet_siders['Tipo'] =current_day_fleet_siders['Tipo de Vehículo']
    #current_day_fleet_siders=current_day_fleet[current_day_fleet['Tipo de Vehículo']=='COLA P.']
    #print(default_start_time)
    current_day_fleet_siders=assign_initial_time(current_day_fleet_siders,fecha_next_str,default_start_time,day)
    current_day_fleet_siders['Ficha']=Fleet_availability['Combinations Final']
    current_day_fleet_siders=current_day_fleet_siders[['Ficha', 'Remolque','Tipo de Vehículo', 'Modelo', 'Capacidad KG', 'Capacidad Ton',
       'Capacidad Pallets', 'Horas de utilidad', 'BREWERY', 'Tipo',
       'Fleet_start_time', 'Fleet_start_date_time',
       'Fleet_start_date_time_org', 'Max_reach_time_at_destiny']]
    #current_day_fleet_siders_CND=current_day_fleet_siders[current_day_fleet_siders.BREWERY=='CND']
    #current_day_fleet_siders_HN=current_day_fleet_siders[current_day_fleet_siders.BREWERY=='HN']
    return current_day_fleet_siders


In [220]:
# Fleet_availability =pd.read_excel(r'Data/Combinations Trucks.xlsx',skiprows=1,sheet_name='Hoja1')
# Fleet_weight_tractor=pd.read_excel(r'Data/Combinations Trucks.xlsx',sheet_name='Tractor Info')
# Fleet_weight_remolque=pd.read_excel(r'Data/Combinations Trucks.xlsx',sheet_name='Remolque Info')
# Fleet_availability=Fleet_availability[['Combinations Final']]
# Fleet_availability=Fleet_availability.dropna(subset = ['Combinations Final'])
# Fleet_availability['Remolque'] =Fleet_availability['Combinations Final'].astype(str).apply(format2)
# Fleet_availability['Tractor'] =Fleet_availability['Combinations Final'].astype(str).apply(format1)
# Fleet_availability=Fleet_availability.merge(Fleet_weight_tractor,left_on='Tractor',right_on='Tractor ID ',how='left').merge(Fleet_weight_remolque[['Remolque_ID', 'Weight_of_Remolque_in_KG']],left_on='Remolque',right_on='Remolque_ID',how='left')
# Fleet_availability=Fleet_availability.merge(df_fleet_list, left_on='Remolque',right_on='Ficha',how='left')
# Fleet_availability['Capacidad KG']=Fleet_availability['Capacidad KG_max']-(Fleet_availability['Weight of tractor in KG ']+Fleet_availability['Weight_of_Remolque_in_KG'])
# Fleet_availability = Fleet_availability.dropna(how='any',axis=0) 

In [221]:
Fleet_availability =pd.read_excel(r'Data/Combinations Trucks.xlsx',skiprows=1,sheet_name='Hoja1')
Fleet_weight_tractor=pd.read_excel(r'Data/Combinations Trucks.xlsx',sheet_name='Tractor Info')
Fleet_weight_remolque=pd.read_excel(r'Data/Combinations Trucks.xlsx',sheet_name='Remolque Info')

In [222]:
current_day_fleet_siders= get_current_date_available_sider(df_fleet_list,Fleet_availability,Fleet_weight_tractor,Fleet_weight_remolque,default_start_time,fecha_str,day).reset_index().drop(['index'],axis=1)

In [223]:
current_day_fleet_siders.head(n=1)

,Ficha,Remolque,Tipo de Vehículo,Modelo,Capacidad KG,Capacidad Ton,Capacidad Pallets,Horas de utilidad,BREWERY,Tipo,Fleet_start_time,Fleet_start_date_time,Fleet_start_date_time_org,Max_reach_time_at_destiny
0,HN-1333 HN-1064,HN-1064,Propia,Loginhsa,21504.576752,41,24,24 Hr,all,Propia,00:00:01,2023-09-14 00:00:01,2023-09-14 00:00:01,2023-09-14 23:00:01


In [224]:
current_day_fleet_siders[current_day_fleet_siders['Ficha']=='HN-VC70  HN-VR96']

,Ficha,Remolque,Tipo de Vehículo,Modelo,Capacidad KG,Capacidad Ton,Capacidad Pallets,Horas de utilidad,BREWERY,Tipo,Fleet_start_time,Fleet_start_date_time,Fleet_start_date_time_org,Max_reach_time_at_destiny
31,HN-VC70 HN-VR96,HN-VR96,Propia,Loginhsa,26054.597192,41,28,24 Hr,all,Propia,00:00:01,2023-09-14 00:00:01,2023-09-14 00:00:01,2023-09-14 23:00:01


Restrictions

In [225]:
def Restrictions_input(current_day_fleet_siders,Restrictions):
    Restrictions=Restrictions1[['Remolque','Restriccion.1','Origen','SKU']]
    Restrictions=Restrictions.dropna(subset=['Remolque'])
    Restrictions['Remolque']='HN-'+Restrictions['Remolque'].astype(str)
    ficha_do_not_use_list=list(Restrictions[Restrictions['Restriccion.1'].isin(['Accidentado','Decomisado'])]['Remolque'].unique())
    current_day_fleet_siders=current_day_fleet_siders[~(current_day_fleet_siders.Ficha.isin(ficha_do_not_use_list))]
    ##### Merge with Fleet data to get final restrictions
    current_day_fleet_siders_Rest=current_day_fleet_siders.merge(Restrictions, left_on='Remolque',right_on='Remolque',how='left').drop(['Remolque'],axis=1).rename(columns={'Restriccion.1':'Ficha_Restriction_destiny','Origen':'Ficha_Restriction_origin','SKU':'Ficha_Restriction_sku'})
    current_day_fleet_siders_Rest['Ficha_Restriction_destiny']=np.where(current_day_fleet_siders_Rest['Ficha_Restriction_destiny'].isin(['Monitor','Ninguna']),'No restriction',np.where(current_day_fleet_siders_Rest['Ficha_Restriction_destiny']=='Dollie','DH00',np.where(current_day_fleet_siders_Rest['Ficha_Restriction_destiny']=='saba','DH11',
    np.where(current_day_fleet_siders_Rest['Ficha_Restriction_destiny']=='Talanga individual','DH10-I',np.where(current_day_fleet_siders_Rest['Ficha_Restriction_destiny']=='Choluteca individual','DH03-I',
    np.where(current_day_fleet_siders_Rest['Ficha_Restriction_destiny']=='Juticalpa individual','DH15-I',np.where(current_day_fleet_siders_Rest['Ficha_Restriction_destiny']=='La Ceiba individual','DH02',
    np.where(current_day_fleet_siders_Rest['Ficha_Restriction_destiny']=='La Granja','DH01',np.where(current_day_fleet_siders_Rest['Ficha_Restriction_destiny']=='Choluteca','DH03',current_day_fleet_siders_Rest['Ficha_Restriction_destiny'])))))))))
                                                                        
    current_day_fleet_siders_Rest['Ficha_Restriction_destiny']=current_day_fleet_siders_Rest['Ficha_Restriction_destiny'].fillna('No restriction')
    #dict_d={'Talanga individual':'DH10-I','Choluteca individual':'DH03-I','Juticalpa individual':'DH15-I','La Ceiba individual':'DH02','La Granja':'DH01','Saba':'DH11','San Pedro Sula':'DH00','Choluteca':'DH03'}
    #current_day_fleet_siders_Rest['Ficha_Restriction_destiny_code']=current_day_fleet_siders_Rest['Ficha_Restriction_destiny'].map(dict_d)
    current_day_fleet_siders_Rest['Ficha_Restriction_origin']=current_day_fleet_siders_Rest['Ficha_Restriction_origin'].fillna('No restriction')
    current_day_fleet_siders_Rest['Ficha_Restriction_sku']=current_day_fleet_siders_Rest['Ficha_Restriction_sku'].fillna('No restriction')
    return current_day_fleet_siders_Rest

In [226]:
Restrictions1 =pd.read_excel('Data/Restricciones de Flota.xlsx',sheet_name='Restricciones',skiprows=2)
current_day_fleet_siders=Restrictions_input(current_day_fleet_siders,Restrictions1)
print(current_day_fleet_siders.shape)
current_day_fleet_siders.head()

(311, 16)


,Ficha,Tipo de Vehículo,Modelo,Capacidad KG,Capacidad Ton,Capacidad Pallets,Horas de utilidad,BREWERY,Tipo,Fleet_start_time,Fleet_start_date_time,Fleet_start_date_time_org,Max_reach_time_at_destiny,Ficha_Restriction_destiny,Ficha_Restriction_origin,Ficha_Restriction_sku
0,HN-1333 HN-1064,Propia,Loginhsa,21504.576752,41,24,24 Hr,all,Propia,00:00:01,2023-09-14 00:00:01,2023-09-14 00:00:01,2023-09-14 23:00:01,DH00,No restriction,No restriction
1,HN-VC28-2 HN-MR06,Propia,Loginhsa,26049.592412,41,28,24 Hr,all,Propia,00:00:01,2023-09-14 00:00:01,2023-09-14 00:00:01,2023-09-14 23:00:01,DH01,No restriction,No restriction
2,HN-MC70 HN-MR27,Propia,Loginhsa,25739.350752,41,28,24 Hr,all,Propia,00:00:01,2023-09-14 00:00:01,2023-09-14 00:00:01,2023-09-14 23:00:01,DH15-I,No restriction,No restriction
3,HN-HC05 HN-MR78,Hibrida,HALL,25926.230656,41,28,24 Hr,all,Hibrida,00:00:01,2023-09-14 00:00:01,2023-09-14 00:00:01,2023-09-14 23:00:01,No restriction,No restriction,No restriction
4,HN-EC15 HN-MR99,Hibrida,TOM,25454.478515,41,28,24 Hr,all,Hibrida,00:00:01,2023-09-14 00:00:01,2023-09-14 00:00:01,2023-09-14 23:00:01,No restriction,No restriction,No restriction


In [227]:
current_day_fleet_siders[current_day_fleet_siders['Ficha_Restriction_destiny']!='No restriction']['Ficha'].shape[0]

139

In [228]:
current_day_fleet_siders[current_day_fleet_siders['Ficha_Restriction_destiny']=='DH00']

,Ficha,Tipo de Vehículo,Modelo,Capacidad KG,Capacidad Ton,Capacidad Pallets,Horas de utilidad,BREWERY,Tipo,Fleet_start_time,Fleet_start_date_time,Fleet_start_date_time_org,Max_reach_time_at_destiny,Ficha_Restriction_destiny,Ficha_Restriction_origin,Ficha_Restriction_sku
0,HN-1333 HN-1064,Propia,Loginhsa,21504.576752,41,24,24 Hr,all,Propia,00:00:01,2023-09-14 00:00:01,2023-09-14 00:00:01,2023-09-14 23:00:01,DH00,No restriction,No restriction
45,HN-1333 HN-1065,Propia,Loginhsa,21504.576752,41,24,24 Hr,all,Propia,00:00:01,2023-09-14 00:00:01,2023-09-14 00:00:01,2023-09-14 23:00:01,DH00,No restriction,No restriction
46,HN-1333 HN-1684,Propia,Loginhsa,21504.576752,41,24,24 Hr,all,Propia,00:00:01,2023-09-14 00:00:01,2023-09-14 00:00:01,2023-09-14 23:00:01,DH00,No restriction,No restriction
47,HN-1333 HN-1685,Propia,Loginhsa,21504.576752,41,24,24 Hr,all,Propia,00:00:01,2023-09-14 00:00:01,2023-09-14 00:00:01,2023-09-14 23:00:01,DH00,No restriction,No restriction


In [229]:
current_day_fleet_siders[current_day_fleet_siders['Ficha']=='HN-MC78  HN-ML34']

,Ficha,Tipo de Vehículo,Modelo,Capacidad KG,Capacidad Ton,Capacidad Pallets,Horas de utilidad,BREWERY,Tipo,Fleet_start_time,Fleet_start_date_time,Fleet_start_date_time_org,Max_reach_time_at_destiny,Ficha_Restriction_destiny,Ficha_Restriction_origin,Ficha_Restriction_sku
37,HN-MC78 HN-ML34,Propia,Loginhsa,25514.806822,41,30,24 Hr,all,Propia,00:00:01,2023-09-14 00:00:01,2023-09-14 00:00:01,2023-09-14 23:00:01,No restriction,No restriction,No restriction


### ATTENTION AND DISTANCE DETAILS 

In [230]:
def format1(str1):
    ##convert time from 1900-01-01 02:00:00	 to 02:00:00
    result=str1
    if len(str1.split(" "))==2:
        result=str1.split(" ")[1]
    return result 

#### Origin

In [231]:
origin_attention_df1 =pd.read_excel('Data/MASTER DATA VENTANA DE ATENCION T2.xlsx',sheet_name='Origen',skiprows=1)

In [232]:
origin_attention_df1.columns

Index(['Turno', 'Inicio', 'Fin', 'T1', 'Unnamed: 4', 'Turno.1', 'Inicio.1',
       'Fin.1', 'T1.1', 'Unnamed: 9', 'Turno.2', 'Inicio.2', 'Fin.2', 'T1.2',
       'Unnamed: 14', 'Turno.3', 'Inicio.3', 'Fin.3', 'T1.3', 'Unnamed: 19',
       'Turno.4', 'Inicio.4', 'Fin.4', 'T1.4'],
      dtype='object')

In [233]:
origin_attention_df1=origin_attention_df1.drop(['Unnamed: 4','Unnamed: 9','Unnamed: 14','Unnamed: 19'],axis=1)
origin_attention_df1['COD']='SH01'
origin_attention_df1['COD.1']='DH22'
origin_attention_df1['COD.2']='DH20'
origin_attention_df1['COD.3']='BH01'
origin_attention_df1['COD.4']='DH19'

origin_dict={'SH01':'CSD','DH22':'KEYMART','DH20':'POLVORIN','BH01':'BEER','DH19':'ALDESA'}
origin_dict

origin_attention_df=origin_attention_df1[['Inicio','Fin','T1','COD']].append(origin_attention_df1[['Inicio.1','Fin.1','T1.1','COD.1']].rename(columns={'Inicio.1':'Inicio','Fin.1':'Fin','T1.1':'T1','COD.1':'COD'})).append(origin_attention_df1[['Inicio.2','Fin.2','T1.2','COD.2']].rename(columns={'Inicio.2':'Inicio','Fin.2':'Fin','T1.2':'T1','COD.2':'COD'})).append(origin_attention_df1[['Inicio.3','Fin.3','T1.3','COD.3']].rename(columns={'Inicio.3':'Inicio','Fin.3':'Fin','T1.3':'T1','COD.3':'COD'})).append(origin_attention_df1[['Inicio.4','Fin.4','T1.4','COD.4']].rename(columns={'Inicio.4':'Inicio','Fin.4':'Fin','T1.4':'T1','COD.4':'COD'}))
origin_attention_df['CDD']=origin_attention_df['COD'].map(origin_dict)

In [234]:
origin_attention_df=origin_attention_df.rename(columns={'Inicio':'Attention_Start','Fin':'Attention_End','T1':'Weekday_attention'})
origin_attention_df=origin_attention_df.dropna(subset=['Attention_Start'])
origin_attention_df=origin_attention_df.fillna(0)
origin_attention_df=origin_attention_df.reset_index()

In [235]:
origin_attention_df['Attention_Start']=origin_attention_df['Attention_Start'].astype(str).apply(format1)
origin_attention_df['Attention_End']=origin_attention_df['Attention_End'].astype(str).apply(format1)  

origin_attention_df['Attention_Start'] = pd.to_datetime(origin_attention_df['Attention_Start'], format='%H:%M:%S').dt.time
origin_attention_df['Attention_End'] = pd.to_datetime(origin_attention_df['Attention_End'], format='%H:%M:%S').dt.time

origin_attention_df['Saturday_attention']=0
origin_attention_df['Sunday_attention']=0

origin_attention_df1=origin_attention_df[['COD','CDD','Attention_Start','Attention_End','Weekday_attention','Saturday_attention','Sunday_attention']]

In [236]:
origin_attention_df.head(n=5)

,index,Attention_Start,Attention_End,Weekday_attention,COD,CDD,Saturday_attention,Sunday_attention
0,0,06:00:00,07:00:00,0.0,SH01,CSD,0,0
1,1,07:00:00,08:00:00,0.0,SH01,CSD,0,0
2,2,08:00:00,09:00:00,6.0,SH01,CSD,0,0
3,3,09:00:00,10:00:00,6.0,SH01,CSD,0,0
4,4,10:00:00,11:00:00,6.0,SH01,CSD,0,0


Defining a few variable, which will be in loop later

In [237]:
# origin = 'BH01'
# destiny = 'DH15'
# #Fleet_start_date_time ='2023-09-14 06:00:01'
## return - whether trip can start or not
## if trip can start when can it start - it can start at 8

#### Destination

In [238]:
destiny_attention_df1_weekday =pd.read_excel('Data/MASTER DATA VENTANA DE ATENCION T2.xlsx',sheet_name='Ventanas T1',skiprows=3)
destiny_attention_df1_weekday=destiny_attention_df1_weekday.head(n=24)

destiny_attention_df1_Sat =pd.read_excel('Data/MASTER DATA VENTANA DE ATENCION T2.xlsx',sheet_name='Ventanas T1',skiprows=33)
destiny_attention_df1_Sat=destiny_attention_df1_Sat.head(n=24)

destiny_attention_df1_Sun =pd.read_excel('Data/MASTER DATA VENTANA DE ATENCION T2.xlsx',sheet_name='Ventanas T1',skiprows=63)
destiny_attention_df1_Sun=destiny_attention_df1_Sun.head(n=24)

In [239]:
destiny_attention_df1_weekday=destiny_attention_df1_weekday.drop(['Unnamed: 4','Unnamed: 9','Unnamed: 14','Unnamed: 19','Unnamed: 24',
                        'Unnamed: 29','Unnamed: 34','Unnamed: 39','Unnamed: 44','Unnamed: 49','Unnamed: 54','Unnamed: 59','Unnamed: 64','Unnamed: 69'],axis=1)
destiny_attention_df1_weekday['COD']='DH01'
destiny_attention_df1_weekday['COD.1']='DH09'
destiny_attention_df1_weekday['COD.2']='DH03'
destiny_attention_df1_weekday['COD.3']='DH15'
destiny_attention_df1_weekday['COD.4']='DH10'
destiny_attention_df1_weekday['COD.5']='DH08'
destiny_attention_df1_weekday['COD.6']='DH13'
destiny_attention_df1_weekday['COD.7']='DH05'
destiny_attention_df1_weekday['COD.8']='DH12'
destiny_attention_df1_weekday['COD.9']='DH02'
destiny_attention_df1_weekday['COD.10']='DH11'
destiny_attention_df1_weekday['COD.11']='DH04'
destiny_attention_df1_weekday['COD.12']='DH06'
destiny_attention_df1_weekday['COD.13']='DH07'
destiny_attention_df1_weekday['COD.14']='DH00'

Dest_dict={'DH01':'La Granja','DH09':'Comayagua','DH03':'Choluteca','DH15':'Juticalpa','DH10':'Talanga','DH08':'Danli',
          'DH13':'La Entrada','DH05':'Santa Rosa','DH12':'Santa Barbara','DH02':'La Ceiba','DH11':'Saba','DH04':'Puerto Cortes','DH06':'Progreso',
          'DH07':'Tela','DH00':'San Pedro Sula'}
print(Dest_dict)

destiny_attention_df_weekday=destiny_attention_df1_weekday[['Inicio','Fin','T1','COD']].append(destiny_attention_df1_weekday[['Inicio.1','Fin.1','T1.1','COD.1']].rename(columns={'Inicio.1':'Inicio','Fin.1':'Fin','T1.1':'T1','COD.1':'COD'})).append(destiny_attention_df1_weekday[['Inicio.2','Fin.2','T1.2','COD.2']].rename(columns={'Inicio.2':'Inicio','Fin.2':'Fin','T1.2':'T1','COD.2':'COD'})).append(destiny_attention_df1_weekday[['Inicio.3','Fin.3','T1.3','COD.3']].rename(columns={'Inicio.3':'Inicio','Fin.3':'Fin','T1.3':'T1','COD.3':'COD'})).append(destiny_attention_df1_weekday[['Inicio.4','Fin.4','T1.4','COD.4']].rename(columns={'Inicio.4':'Inicio','Fin.4':'Fin','T1.4':'T1','COD.4':'COD'})).append(destiny_attention_df1_weekday[['Inicio.5','Fin.5','T1.5','COD.5']].rename(columns={'Inicio.5':'Inicio','Fin.5':'Fin','T1.5':'T1','COD.5':'COD'})).append(destiny_attention_df1_weekday[['Inicio.6','Fin.6','T1.6','COD.6']].rename(columns={'Inicio.6':'Inicio','Fin.6':'Fin','T1.6':'T1','COD.6':'COD'})).append(destiny_attention_df1_weekday[['Inicio.7','Fin.7','T1.7','COD.7']].rename(columns={'Inicio.7':'Inicio','Fin.7':'Fin','T1.7':'T1','COD.7':'COD'})).append(destiny_attention_df1_weekday[['Inicio.8','Fin.8','T1.8','COD.8']].rename(columns={'Inicio.8':'Inicio','Fin.8':'Fin','T1.8':'T1','COD.8':'COD'})).append(destiny_attention_df1_weekday[['Inicio.9','Fin.9','T1.9','COD.9']].rename(columns={'Inicio.9':'Inicio','Fin.9':'Fin','T1.9':'T1','COD.9':'COD'})).append(destiny_attention_df1_weekday[['Inicio.10','Fin.10','T1.10','COD.10']].rename(columns={'Inicio.10':'Inicio','Fin.10':'Fin','T1.10':'T1','COD.10':'COD'})).append(destiny_attention_df1_weekday[['Inicio.11','Fin.11','T1.11','COD.11']].rename(columns={'Inicio.11':'Inicio','Fin.11':'Fin','T1.11':'T1','COD.11':'COD'})).append(destiny_attention_df1_weekday[['Inicio.12','Fin.12','T1.12','COD.12']].rename(columns={'Inicio.12':'Inicio','Fin.12':'Fin','T1.12':'T1','COD.12':'COD'})).append(destiny_attention_df1_weekday[['Inicio.13','Fin.13','T1.13','COD.13']].rename(columns={'Inicio.13':'Inicio','Fin.13':'Fin','T1.13':'T1','COD.13':'COD'})).append(destiny_attention_df1_weekday[['Inicio.14','Fin.14','T1.14','COD.14']].rename(columns={'Inicio.14':'Inicio','Fin.14':'Fin','T1.14':'T1','COD.14':'COD'}))
destiny_attention_df_weekday['CDD']=destiny_attention_df_weekday['COD'].map(Dest_dict)

destiny_attention_df_weekday=destiny_attention_df_weekday.rename(columns={'Inicio':'Attention_Start','Fin':'Attention_End','T1':'Weekday_attention'})
destiny_attention_df_weekday=destiny_attention_df_weekday.dropna(subset=['Attention_Start'])
destiny_attention_df_weekday=destiny_attention_df_weekday.fillna(0)
destiny_attention_df_weekday=destiny_attention_df_weekday.reset_index()

destiny_attention_df_weekday['Attention_Start']=destiny_attention_df_weekday['Attention_Start'].astype(str).apply(format1)
destiny_attention_df_weekday['Attention_End']=destiny_attention_df_weekday['Attention_End'].astype(str).apply(format1)  
destiny_attention_df_weekday['Attention_Start'] = pd.to_datetime(destiny_attention_df_weekday['Attention_Start'], format='%H:%M:%S').dt.time
destiny_attention_df_weekday['Attention_End'] = pd.to_datetime(destiny_attention_df_weekday['Attention_End'], format='%H:%M:%S').dt.time
destiny_attention_df_weekday=destiny_attention_df_weekday[['COD','CDD','Attention_Start','Attention_End','Weekday_attention']]

{'DH01': 'La Granja', 'DH09': 'Comayagua', 'DH03': 'Choluteca', 'DH15': 'Juticalpa', 'DH10': 'Talanga', 'DH08': 'Danli', 'DH13': 'La Entrada', 'DH05': 'Santa Rosa', 'DH12': 'Santa Barbara', 'DH02': 'La Ceiba', 'DH11': 'Saba', 'DH04': 'Puerto Cortes', 'DH06': 'Progreso', 'DH07': 'Tela', 'DH00': 'San Pedro Sula'}


In [240]:
destiny_attention_df1_Sat=destiny_attention_df1_Sat.drop(['Unnamed: 4','Unnamed: 9','Unnamed: 14','Unnamed: 19','Unnamed: 24',
                        'Unnamed: 29','Unnamed: 34','Unnamed: 39','Unnamed: 44','Unnamed: 49','Unnamed: 54','Unnamed: 59','Unnamed: 64','Unnamed: 69'],axis=1)
destiny_attention_df1_Sat['COD']='DH01'
destiny_attention_df1_Sat['COD.1']='DH09'
destiny_attention_df1_Sat['COD.2']='DH03'
destiny_attention_df1_Sat['COD.3']='DH15'
destiny_attention_df1_Sat['COD.4']='DH10'
destiny_attention_df1_Sat['COD.5']='DH08'
destiny_attention_df1_Sat['COD.6']='DH13'
destiny_attention_df1_Sat['COD.7']='DH05'
destiny_attention_df1_Sat['COD.8']='DH12'
destiny_attention_df1_Sat['COD.9']='DH02'
destiny_attention_df1_Sat['COD.10']='DH11'
destiny_attention_df1_Sat['COD.11']='DH04'
destiny_attention_df1_Sat['COD.12']='DH06'
destiny_attention_df1_Sat['COD.13']='DH07'
destiny_attention_df1_Sat['COD.14']='DH00'

Dest_dict={'DH01':'La Granja','DH09':'Comayagua','DH03':'Choluteca','DH15':'Juticalpa','DH10':'Talanga','DH08':'Danli',
          'DH13':'La Entrada','DH05':'Santa Rosa','DH12':'Santa Barbara','DH02':'La Ceiba','DH11':'Saba','DH04':'Puerto Cortes','DH06':'Progreso',
          'DH07':'Tela','DH00':'San Pedro Sula'}
print(Dest_dict)

destiny_attention_df_Sat=destiny_attention_df1_Sat[['Inicio','Fin','T1','COD']].append(destiny_attention_df1_Sat[['Inicio.1','Fin.1','T1.1','COD.1']].rename(columns={'Inicio.1':'Inicio','Fin.1':'Fin','T1.1':'T1','COD.1':'COD'})).append(destiny_attention_df1_Sat[['Inicio.2','Fin.2','T1.2','COD.2']].rename(columns={'Inicio.2':'Inicio','Fin.2':'Fin','T1.2':'T1','COD.2':'COD'})).append(destiny_attention_df1_Sat[['Inicio.3','Fin.3','T1.3','COD.3']].rename(columns={'Inicio.3':'Inicio','Fin.3':'Fin','T1.3':'T1','COD.3':'COD'})).append(destiny_attention_df1_Sat[['Inicio.4','Fin.4','T1.4','COD.4']].rename(columns={'Inicio.4':'Inicio','Fin.4':'Fin','T1.4':'T1','COD.4':'COD'})).append(destiny_attention_df1_Sat[['Inicio.5','Fin.5','T1.5','COD.5']].rename(columns={'Inicio.5':'Inicio','Fin.5':'Fin','T1.5':'T1','COD.5':'COD'})).append(destiny_attention_df1_Sat[['Inicio.6','Fin.6','T1.6','COD.6']].rename(columns={'Inicio.6':'Inicio','Fin.6':'Fin','T1.6':'T1','COD.6':'COD'})).append(destiny_attention_df1_Sat[['Inicio.7','Fin.7','T1.7','COD.7']].rename(columns={'Inicio.7':'Inicio','Fin.7':'Fin','T1.7':'T1','COD.7':'COD'})).append(destiny_attention_df1_Sat[['Inicio.8','Fin.8','T1.8','COD.8']].rename(columns={'Inicio.8':'Inicio','Fin.8':'Fin','T1.8':'T1','COD.8':'COD'})).append(destiny_attention_df1_Sat[['Inicio.9','Fin.9','T1.9','COD.9']].rename(columns={'Inicio.9':'Inicio','Fin.9':'Fin','T1.9':'T1','COD.9':'COD'})).append(destiny_attention_df1_Sat[['Inicio.10','Fin.10','T1.10','COD.10']].rename(columns={'Inicio.10':'Inicio','Fin.10':'Fin','T1.10':'T1','COD.10':'COD'})).append(destiny_attention_df1_Sat[['Inicio.11','Fin.11','T1.11','COD.11']].rename(columns={'Inicio.11':'Inicio','Fin.11':'Fin','T1.11':'T1','COD.11':'COD'})).append(destiny_attention_df1_Sat[['Inicio.12','Fin.12','T1.12','COD.12']].rename(columns={'Inicio.12':'Inicio','Fin.12':'Fin','T1.12':'T1','COD.12':'COD'})).append(destiny_attention_df1_Sat[['Inicio.13','Fin.13','T1.13','COD.13']].rename(columns={'Inicio.13':'Inicio','Fin.13':'Fin','T1.13':'T1','COD.13':'COD'})).append(destiny_attention_df1_Sat[['Inicio.14','Fin.14','T1.14','COD.14']].rename(columns={'Inicio.14':'Inicio','Fin.14':'Fin','T1.14':'T1','COD.14':'COD'}))
destiny_attention_df_Sat['CDD']=destiny_attention_df_Sat['COD'].map(Dest_dict)

destiny_attention_df_Sat=destiny_attention_df_Sat.rename(columns={'Inicio':'Attention_Start','Fin':'Attention_End','T1':'Sat_attention'})
destiny_attention_df_Sat=destiny_attention_df_Sat.dropna(subset=['Attention_Start'])
destiny_attention_df_Sat=destiny_attention_df_Sat.fillna(0)
destiny_attention_df_Sat=destiny_attention_df_Sat.reset_index()

destiny_attention_df_Sat['Attention_Start']=destiny_attention_df_Sat['Attention_Start'].astype(str).apply(format1)
destiny_attention_df_Sat['Attention_End']=destiny_attention_df_Sat['Attention_End'].astype(str).apply(format1)  
destiny_attention_df_Sat['Attention_Start'] = pd.to_datetime(destiny_attention_df_Sat['Attention_Start'], format='%H:%M:%S').dt.time
destiny_attention_df_Sat['Attention_End'] = pd.to_datetime(destiny_attention_df_Sat['Attention_End'], format='%H:%M:%S').dt.time
destiny_attention_df_Sat=destiny_attention_df_Sat[['COD','CDD','Attention_Start','Attention_End','Sat_attention']]

{'DH01': 'La Granja', 'DH09': 'Comayagua', 'DH03': 'Choluteca', 'DH15': 'Juticalpa', 'DH10': 'Talanga', 'DH08': 'Danli', 'DH13': 'La Entrada', 'DH05': 'Santa Rosa', 'DH12': 'Santa Barbara', 'DH02': 'La Ceiba', 'DH11': 'Saba', 'DH04': 'Puerto Cortes', 'DH06': 'Progreso', 'DH07': 'Tela', 'DH00': 'San Pedro Sula'}


In [241]:
destiny_attention_df1_Sun.columns

Index(['Turno', 'Inicio', 'Fin', 'T1', 'Unnamed: 4', 'Turno.1', 'Inicio.1',
       'Fin.1', 'T1.1', 'Unnamed: 9', 'Turno.2', 'Inicio.2', 'Fin.2', 'T1.2',
       'Unnamed: 14', 'Turno.3', 'Inicio.3', 'Fin.3', 'T1.3', 'Unnamed: 19',
       'Turno.4', 'Inicio.4', 'Fin.4', 'T1.4', 'Unnamed: 24', 'Turno.5',
       'Inicio.5', 'Fin.5', 'T1.5', 'Unnamed: 29', 'Turno.6', 'Inicio.6',
       'Fin.6', 'T1.6', 'Unnamed: 34', 'Turno.7', 'Inicio.7', 'Fin.7', 'T1.7',
       'Unnamed: 39', 'Turno.8', 'Inicio.8', 'Fin.8', 'T1.8', 'Unnamed: 44',
       'Turno.9', 'Inicio.9', 'Fin.9', 'T1.9', 'Unnamed: 49', 'Turno.10',
       'Inicio.10', 'Fin.10', 'T1.10', 'Unnamed: 54', 'Turno.11', 'Inicio.11',
       'Fin.11', 'T1.11', 'Unnamed: 59', 'Turno.12', 'Inicio.12', 'Fin.12',
       'T1.12', 'Unnamed: 64', 'Turno.13', 'Inicio.13', 'Fin.13', 'T1.13',
       'Unnamed: 69', 'Unnamed: 70', 'Unnamed: 71', 'Unnamed: 72',
       'Unnamed: 73'],
      dtype='object')

In [242]:
destiny_attention_df1_Sun=destiny_attention_df1_Sun.drop(['Unnamed: 4','Unnamed: 9','Unnamed: 14','Unnamed: 19','Unnamed: 24',
                        'Unnamed: 29','Unnamed: 34','Unnamed: 39','Unnamed: 44','Unnamed: 49','Unnamed: 54','Unnamed: 59','Unnamed: 64','Unnamed: 69','Unnamed: 70', 'Unnamed: 71', 'Unnamed: 72',
       'Unnamed: 73'],axis=1)
destiny_attention_df1_Sun['COD']='DH01'
destiny_attention_df1_Sun['COD.1']='DH09'
destiny_attention_df1_Sun['COD.2']='DH03'
destiny_attention_df1_Sun['COD.3']='DH15'
destiny_attention_df1_Sun['COD.4']='DH10'
destiny_attention_df1_Sun['COD.5']='DH08'
destiny_attention_df1_Sun['COD.6']='DH13'
destiny_attention_df1_Sun['COD.7']='DH05'
destiny_attention_df1_Sun['COD.8']='DH12'
destiny_attention_df1_Sun['COD.9']='DH02'
destiny_attention_df1_Sun['COD.10']='DH11'
destiny_attention_df1_Sun['COD.11']='DH04'
destiny_attention_df1_Sun['COD.12']='DH06'
destiny_attention_df1_Sun['COD.13']='DH00'

Dest_dict={'DH01':'La Granja','DH09':'Comayagua','DH03':'Choluteca','DH15':'Juticalpa','DH10':'Talanga','DH08':'Danli',
          'DH13':'La Entrada','DH05':'Santa Rosa','DH12':'Santa Barbara','DH02':'La Ceiba','DH11':'Saba','DH04':'Puerto Cortes','DH06':'Progreso',
          'DH00':'San Pedro Sula'}
print(Dest_dict)

destiny_attention_df_Sun=destiny_attention_df1_Sun[['Inicio','Fin','T1','COD']].append(destiny_attention_df1_Sun[['Inicio.1','Fin.1','T1.1','COD.1']].rename(columns={'Inicio.1':'Inicio','Fin.1':'Fin','T1.1':'T1','COD.1':'COD'})).append(destiny_attention_df1_Sun[['Inicio.2','Fin.2','T1.2','COD.2']].rename(columns={'Inicio.2':'Inicio','Fin.2':'Fin','T1.2':'T1','COD.2':'COD'})).append(destiny_attention_df1_Sun[['Inicio.3','Fin.3','T1.3','COD.3']].rename(columns={'Inicio.3':'Inicio','Fin.3':'Fin','T1.3':'T1','COD.3':'COD'})).append(destiny_attention_df1_Sun[['Inicio.4','Fin.4','T1.4','COD.4']].rename(columns={'Inicio.4':'Inicio','Fin.4':'Fin','T1.4':'T1','COD.4':'COD'})).append(destiny_attention_df1_Sun[['Inicio.5','Fin.5','T1.5','COD.5']].rename(columns={'Inicio.5':'Inicio','Fin.5':'Fin','T1.5':'T1','COD.5':'COD'})).append(destiny_attention_df1_Sun[['Inicio.6','Fin.6','T1.6','COD.6']].rename(columns={'Inicio.6':'Inicio','Fin.6':'Fin','T1.6':'T1','COD.6':'COD'})).append(destiny_attention_df1_Sun[['Inicio.7','Fin.7','T1.7','COD.7']].rename(columns={'Inicio.7':'Inicio','Fin.7':'Fin','T1.7':'T1','COD.7':'COD'})).append(destiny_attention_df1_Sun[['Inicio.8','Fin.8','T1.8','COD.8']].rename(columns={'Inicio.8':'Inicio','Fin.8':'Fin','T1.8':'T1','COD.8':'COD'})).append(destiny_attention_df1_Sun[['Inicio.9','Fin.9','T1.9','COD.9']].rename(columns={'Inicio.9':'Inicio','Fin.9':'Fin','T1.9':'T1','COD.9':'COD'})).append(destiny_attention_df1_Sun[['Inicio.10','Fin.10','T1.10','COD.10']].rename(columns={'Inicio.10':'Inicio','Fin.10':'Fin','T1.10':'T1','COD.10':'COD'})).append(destiny_attention_df1_Sun[['Inicio.11','Fin.11','T1.11','COD.11']].rename(columns={'Inicio.11':'Inicio','Fin.11':'Fin','T1.11':'T1','COD.11':'COD'})).append(destiny_attention_df1_Sun[['Inicio.12','Fin.12','T1.12','COD.12']].rename(columns={'Inicio.12':'Inicio','Fin.12':'Fin','T1.12':'T1','COD.12':'COD'})).append(destiny_attention_df1_Sun[['Inicio.13','Fin.13','T1.13','COD.13']].rename(columns={'Inicio.13':'Inicio','Fin.13':'Fin','T1.13':'T1','COD.13':'COD'}))
destiny_attention_df_Sun['CDD']=destiny_attention_df_Sun['COD'].map(Dest_dict)

destiny_attention_df_Sun=destiny_attention_df_Sun.rename(columns={'Inicio':'Attention_Start','Fin':'Attention_End','T1':'Sun_attention'})
destiny_attention_df_Sun=destiny_attention_df_Sun.dropna(subset=['Attention_Start'])
destiny_attention_df_Sun=destiny_attention_df_Sun.fillna(0)
destiny_attention_df_Sun=destiny_attention_df_Sun.reset_index()

destiny_attention_df_Sun['Attention_Start']=destiny_attention_df_Sun['Attention_Start'].astype(str).apply(format1)
destiny_attention_df_Sun['Attention_End']=destiny_attention_df_Sun['Attention_End'].astype(str).apply(format1)  
destiny_attention_df_Sun['Attention_Start'] = pd.to_datetime(destiny_attention_df_Sun['Attention_Start'], format='%H:%M:%S').dt.time
destiny_attention_df_Sun['Attention_End'] = pd.to_datetime(destiny_attention_df_Sun['Attention_End'], format='%H:%M:%S').dt.time
destiny_attention_df_Sun=destiny_attention_df_Sun[['COD','CDD','Attention_Start','Attention_End','Sun_attention']]

{'DH01': 'La Granja', 'DH09': 'Comayagua', 'DH03': 'Choluteca', 'DH15': 'Juticalpa', 'DH10': 'Talanga', 'DH08': 'Danli', 'DH13': 'La Entrada', 'DH05': 'Santa Rosa', 'DH12': 'Santa Barbara', 'DH02': 'La Ceiba', 'DH11': 'Saba', 'DH04': 'Puerto Cortes', 'DH06': 'Progreso', 'DH00': 'San Pedro Sula'}


In [243]:
destiny_attention_df=destiny_attention_df_weekday.merge(destiny_attention_df_Sat, on=['COD','CDD','Attention_Start','Attention_End'],how='left').merge(destiny_attention_df_Sun, on=['COD','CDD','Attention_Start','Attention_End'],how='left')
destiny_attention_df=destiny_attention_df.fillna(0)

#### Time taken

In [244]:
def convert_hr_min_to_minutes(str1):
    ## 04:30	 to 270 mins
    hour1=str1.split(":")[0]
    min1=str1.split(":")[1]
    hour1_min=int(hour1)*60
    total_min=hour1_min+int(min1)
    return total_min

In [245]:
Time_df =pd.read_excel('Data/Inputs - Static.xlsx',sheet_name='Tiempos',skiprows=2)

In [246]:
### For 12 columns 
distance_and_time =pd.DataFrame(index=np.arange(1), columns=np.arange(12))
distance_and_time=distance_and_time.rename(columns={0: 'Origin',1: 'Origin_Code',2: 'Destiny',3: 'Destiny_Code',4: 'Clave',5:'time_one_way_min',6:'time_two_way_min',7:'Loading_time_origin_min',8:'unloading_loading_time_in_dest_min',9:'row_time_origin_min',10:'waiting_time_origin_min',11:'break_time_min'})

##### Put the Individual list in time itself

origin_dict={'SH01':'CSD','DH22':'KEYMART','DH20':'POLVORIN','BH01':'BEER','DH19':'ALDESA'}
Dest_dict={'DH01':'La Granja','DH09':'Comayagua','DH03':'Choluteca','DH15':'Juticalpa','DH10':'Talanga','DH08':'Danli',
          'DH13':'La Entrada','DH05':'Santa Rosa','DH12':'Santa Barbara','DH02':'La Ceiba','DH11':'Saba','DH04':'Puerto Cortes','DH06':'Progreso',
          'DH00':'San Pedro Sula','DH07':'Tela','DH14':'Roatan','DH03-I':'Choluteca Individual','DH10-I':'Talanga Individual','DH15-I':'Juticalpa Individual'}

distance_and_time1 =pd.DataFrame()
for i in list(origin_dict.keys()):
    distance_and_time['Origin']=origin_dict[i]
    distance_and_time['Origin_Code']=i
    for j in list(Dest_dict.keys()):
        distance_and_time['Destiny']=Dest_dict[j]
        distance_and_time['Destiny_Code']=j
        distance_and_time1=distance_and_time1.append(distance_and_time)
        
distance_and_time1['Clave']= distance_and_time1['Origin'].str.upper()+"-"+distance_and_time1['Destiny'].str.upper()
distance_and_time1=distance_and_time1.fillna(0)

distance_and_time2 =pd.DataFrame()
for i in range(Time_df.shape[0]):
    #print(i)
    time_dest_temp=distance_and_time1[distance_and_time1['Destiny']==Time_df.iloc[i]['Unnamed: 0']]
    time_dest_temp['waiting_time_origin_min']=Time_df.iloc[i]['TC']
    time_dest_temp['Loading_time_origin_min']=Time_df.iloc[i]['TP']
    time_dest_temp['row_time_origin_min']=Time_df.iloc[i]['TF']
    time_dest_temp['time_one_way_min']=Time_df.iloc[i]['TT']
    time_dest_temp['unloading_loading_time_in_dest_min']=Time_df.iloc[i]['TCD']
    time_dest_temp['break_time_min']=Time_df.iloc[i]['TD']
    time_dest_temp['time_two_way_min']=Time_df.iloc[i]['TR']
    distance_and_time2=distance_and_time2.append(time_dest_temp)
    
time_cols=['time_one_way_min', 'time_two_way_min', 'Loading_time_origin_min',
       'unloading_loading_time_in_dest_min', 'row_time_origin_min',
       'waiting_time_origin_min', 'break_time_min']

for i in time_cols:
    #print(i)
    distance_and_time2[i]=distance_and_time2[i].astype(str).apply(format2)
    distance_and_time2[i]=distance_and_time2[i].astype(str).apply(convert_hr_min_to_minutes)
distance_and_time2['Loading_time_origin_min'] =distance_and_time2['waiting_time_origin_min']+distance_and_time2['Loading_time_origin_min']+distance_and_time2['row_time_origin_min']
    
distance_and_time=distance_and_time2

In [247]:
#distance_and_time[distance_and_time['Destiny_Code']=='DH15']

### Distribution Plan

In [248]:
sku_description.columns

Index(['codigo', 'Description', 'Size', 'Pack', 'Category', 'Brand',
       'Box x Hls', 'package_per_pallet', 'Weight x Box', 'Units x Box',
       'Active Code', 'pound_weight_per_pallet', 'HLS x Box',
       'kg_weight_per_pallet'],
      dtype='object')

In [249]:
Dist_plan =pd.read_excel('Data/Space and Optimus DP.xlsx')
Dist_plan['Fecha']=pd.to_datetime(Dist_plan['Fecha'])

Dist_plan1=Dist_plan[Dist_plan['Fecha']==pd.to_datetime(fecha_str, format='%d-%m-%Y')]
Dist_plan1=Dist_plan1.drop(['Fecha'],axis=1)
print(Dist_plan1.shape)
Dist_plan1=Dist_plan1.rename(columns={'Origen': 'Origin','Destino': 'Destination','Total': 'Total_boxes','sku_SAP': 'codigo'})
Dist_plan1 =Dist_plan1.merge(sku_description[['codigo','kg_weight_per_pallet','package_per_pallet']], on ='codigo', how ='left')
Dist_plan1['Total_pallets']=Dist_plan1['Total_boxes']/Dist_plan1['package_per_pallet']
Dist_plan1=Dist_plan1[Dist_plan1['Destination']!='DH14']


Dist_plan1['org-priority']=Dist_plan1['Origin']+"-"+Dist_plan1['Prioridad'].astype(str)
#print(Dist_plan1.head(50))
#code modified by Jitendra as sort is not happening properly as it is happening in excel sheet 
#commented 
# Dist_plan1=Dist_plan1.sort_values(['Prioridad'],ascending=[True])
#added 
Dist_plan1_1=Dist_plan1[(Dist_plan1.Prioridad==1)]
Dist_plan1_2=Dist_plan1[(Dist_plan1.Prioridad==2)]
Dist_plan1_3=Dist_plan1[(Dist_plan1.Prioridad==3)]
print(Dist_plan1.shape)
Dist_plan1=pd.concat([Dist_plan1_1,Dist_plan1_2,Dist_plan1_3])
print(Dist_plan1.shape)
#Question for Sejuti what to do if there is no priority for a row 
# print(Dist_plan1.head(10))
Dist_plan2 =Dist_plan1.copy(deep=True)

All_sku=list(Dist_plan1['codigo'].unique())

All_destiny=list(Dist_plan1['Destination'].unique())
print('total destiny',len(All_destiny))

(366, 5)
(343, 9)
(343, 9)
total destiny 15


In [250]:
#Dist_plan1.head(50)

#### Origin-Dest Before Melt DF for Jitendra

In [251]:
Dist_plan1.columns

Index(['Origin', 'Destination', 'codigo', 'Prioridad', 'Total_boxes',
       'kg_weight_per_pallet', 'package_per_pallet', 'Total_pallets',
       'org-priority'],
      dtype='object')

In [252]:
sku_description.columns

Index(['codigo', 'Description', 'Size', 'Pack', 'Category', 'Brand',
       'Box x Hls', 'package_per_pallet', 'Weight x Box', 'Units x Box',
       'Active Code', 'pound_weight_per_pallet', 'HLS x Box',
       'kg_weight_per_pallet'],
      dtype='object')

In [253]:
OrgPr1={}
for i in list(Dist_plan1['Origin'].unique()):
    #print(i)
    temp = Dist_plan1[Dist_plan1['Origin']==i]
    OrgPr1[i]=temp
    
OrgPrBefMelt1={}
for i in list(OrgPr1.keys()):
    #print(i)
    temp1=OrgPr1[i].pivot(index='codigo', columns='Destination', values='Total_pallets').fillna(0).reset_index().rename_axis(None, axis=1)
    OrgPrBefMelt1[i]=temp1
    #### Ensuring all datasets(all or-dest-priority comb) have the exhaustive list of destinities-total 15.
    diff_col=list(set(All_destiny).difference(list(OrgPrBefMelt1[i].columns)))
    for l in diff_col:
        OrgPrBefMelt1[i][l]=0
        
# Ensuring all datasets(all or-dest-priority comb) have the exhaustive list of SKUs.
for i in OrgPrBefMelt1.keys():
    OrgPrBefMelt1[i]=OrgPrBefMelt1[i].append(pd.DataFrame(list(set(All_sku).difference(list(OrgPrBefMelt1[i]['codigo'].unique()))), columns=['codigo'])).fillna(0).reset_index().drop(['index'],axis=1)
    OrgPrBefMelt1[i]=OrgPrBefMelt1[i].merge(sku_description[['codigo','kg_weight_per_pallet']])

In [254]:
for i in OrgPrBefMelt1.keys():
    print(OrgPrBefMelt1[i].shape)

(43, 17)
(43, 17)
(43, 17)


#### Creating Origin Priority dataframe

In [255]:
OrgPr={}
for i in list(Dist_plan1['org-priority'].unique()):
    #print(i)
    temp = Dist_plan1[Dist_plan1['org-priority']==i]
    OrgPr[i]=temp
    
OrgPrBefMelt={}
for i in list(OrgPr.keys()):
    #print(i)
    temp1=OrgPr[i].pivot(index='codigo', columns='Destination', values='Total_pallets').fillna(0).reset_index().rename_axis(None, axis=1)
    OrgPrBefMelt[i]=temp1
    #### Ensuring all datasets(all or-dest-priority comb) have the exhaustive list of destinities-total 15.
    diff_col=list(set(All_destiny).difference(list(OrgPrBefMelt[i].columns)))
    for l in diff_col:
        OrgPrBefMelt[i][l]=0
    
    
    # Ensuring all datasets(all or-dest-priority comb) have the exhaustive list of SKUs.
for i in OrgPrBefMelt.keys():
    OrgPrBefMelt[i]=OrgPrBefMelt[i].append(pd.DataFrame(list(set(All_sku).difference(list(OrgPrBefMelt[i]['codigo'].unique()))), columns=['codigo'])).fillna(0).reset_index().drop(['index'],axis=1)

### Origin Attention check

In [256]:
def get_attention_hour_list_v1(origin_attention_df,org,start_time_p,attention_list_allowed,attention_dict_allowed): 
    #get origin attention hour list which is 24 hr or max time time it can start from origin 
    
    origin_attention_df_1=origin_attention_df[origin_attention_df.COD==org]
    origin_attention_df_1=origin_attention_df_1.sort_values(by='Attention_Start')
    allowed_attention_list=attention_list_allowed
    allowed_attention_dict=attention_dict_allowed
    attention_hour_list=[]
    from itertools import cycle
    pool = cycle(origin_attention_df_1['Attention_Start'].values)
    start_time=start_time_p
    count=0
    for ele in pool:
        if count==36:
            break
        count=count+1
        attention_hour_list.append(str(ele))
    start_index=attention_hour_list.index(start_time)
    if start_time_p not in allowed_attention_list:
        attention_hour_list=[]
    else:
        num=allowed_attention_dict[start_time_p]
        end_index=start_index+num    

        attention_hour_list=attention_hour_list[start_index:end_index+1]


    return attention_hour_list

In [257]:
def get_attention_hour_list(origin_attention_df,org,start_time_p): 
    #get destination attention hour to check list which is 12 hr from origin attention obtained
    origin_attention_df_1=origin_attention_df[origin_attention_df.COD==org]
    #print("dest_atten",origin_attention_df_1)
    origin_attention_df_1=origin_attention_df_1.sort_values(by='Attention_Start')
    attention_hour_list=[]
    from itertools import cycle
    pool = cycle(origin_attention_df_1['Attention_Start'].values)
    #print(origin_attention_df_1['Attention_Start'].values)
    #print("pool",pool)
    start_time=start_time_p
    #print("inside get_attnhourlist",start_time)
    count=0
    for ele in pool:
        #print(ele)
        if count==36:
            break
        count=count+1
        attention_hour_list.append(str(ele))
    #print("attention_hour_list",attention_hour_list)
    start_index=attention_hour_list.index(start_time)
    end_index=start_index+12    

    attention_hour_list=attention_hour_list[start_index:end_index+1]

    return attention_hour_list	

In [258]:
Max_reach_time_at_destiny_p=str(current_day_fleet_siders.loc[0,'Max_reach_time_at_destiny'].time())[0:5]+":"+"00"
fleet_start_date_time = current_day_fleet_siders.loc[0,'Fleet_start_date_time']
fleet_start_time=str(fleet_start_date_time.hour).zfill(2)+":00:00"

In [259]:
night_time_dict={'00:00:00':23,'01:00:00':22,'02:00:00':21,'03:00:00':20,'04:00:00':19,'05:00:00':18,'06:00:00':17,'07:00:00':16,'08:00:00':15,'09:00:00':14,'10:00:00':13,
                '11:00:00':12,'12:00:00':11,'13:00:00':10,'14:00:00':9,'15:00:00':8,'16:00:00':7,'17:00:00':6,'18:00:00':5,'19:00:00':4,'20:00:00':3,'21:00:00':2,'22:00:00':1,'23:00:00':0}
night_time_attention_allowed=[]
for ele in night_time_dict.keys():
    night_time_attention_allowed.append(ele)
#night_time_attention_allowed

In [260]:
attention_list_allowed=night_time_attention_allowed
attention_dict_allowed =night_time_dict

In [261]:
def check_origin_attention_v1(origin_attention_df,origin,fleet_start_time_p,Max_reach_time_at_destiny_p,day_of_week,attention_list_allowed,attention_dict_allowed):
	#check origin attention and return when can we start the sider 
	
    origin_attention_df_temp=origin_attention_df[["COD","Attention_Start","Attention_End",day_of_week+"_attention"]]
    origin_attention_df_temp=origin_attention_df_temp[origin_attention_df_temp.COD==origin]
    origin_attention_df_temp['Attention_Start']=origin_attention_df_temp['Attention_Start'].astype(str)
    fleet_start_time=fleet_start_time_p
    Max_reach_time_at_destiny=Max_reach_time_at_destiny_p
    truck_sent =False
    attention_start_obtained="NA"
#     from itertools import cycle
#     pool = cycle(origin_attention_df_temp['Attention_Start'].values).tolist()
#     start_index=origin_attention_df_temp.Attention_Start.values.tolist().index(fleet_start_time_p)
#     end_index=start_index+iteration_checking
    #print(origin,fleet_start_time)
    attention_hour_list=get_attention_hour_list_v1(origin_attention_df,origin,fleet_start_time,attention_list_allowed,attention_dict_allowed)
    #print("origin attention hour list",attention_hour_list)
    if len(attention_hour_list)>0:
        for attention_start  in attention_hour_list:
            if attention_start==Max_reach_time_at_destiny:
                #print("max time reached trip can not be planned")
                break
            #print("currently checking for",attention_start)
            attention_available=origin_attention_df_temp[(origin_attention_df_temp['Attention_Start']==attention_start)][day_of_week+"_attention"].values[0]
            #print("Attention available ",attention_available)
            if attention_available>0:
                attention_start_obtained=attention_start
                truck_sent=True
                break
#     print("attention_start_obtained",attention_start_obtained)
#     print("can we send truck ",truck_sent)
    return attention_start_obtained,truck_sent


In [262]:
# for j in range(current_day_fleet_siders.reset_index(drop=True).shape[0]):
#     Max_reach_time_at_destiny_p=str(current_day_fleet_siders.loc[j,'Max_reach_time_at_destiny'].time())[0:5]+":"+"00"
#     fleet_start_date_time = current_day_fleet_siders.loc[j,'Fleet_start_date_time']
#     fleet_start_time=str(fleet_start_date_time.hour).zfill(2)+":00:00"
#     check_origin_attention_v1(origin_attention_df,origin,fleet_start_time,Max_reach_time_at_destiny_p,day_of_week,attention_list_allowed,attention_dict_allowed)

In [263]:
# j=0
# Max_reach_date_time_at_destiny=current_day_fleet_siders.loc[j,'Max_reach_time_at_destiny']
# fleet_start_date_time = current_day_fleet_siders.loc[j,'Fleet_start_date_time']
# fleet_start_time=str(fleet_start_date_time.hour).zfill(2)+":00:00"
# attention_start_obtained,truck_sent=check_origin_attention_v1(origin_attention_df,origin,fleet_start_time,Max_reach_time_at_destiny_p,day_of_week,attention_list_allowed,attention_dict_allowed)

In [264]:
# distance_and_time['Destiny_Code'].unique()

In [265]:
# current_day_fleet_siders.shape

### Destination Attention check

In [266]:
# Max_reach_date_time_at_destiny_p=Max_reach_date_time_at_destiny
# attention_start_time_obtained=attention_start_obtained

In [267]:
def check_destination_attention_v1(destiny_attention_df,distance_and_time,origin,destiny,attention_start_time_obtained,Max_reach_time_at_destiny_p,fecha_str,day_of_week):
#check destination attention hour
#convert truck_start_time_obtained to date time 
    import datetime
    from datetime import timedelta
    fecha_next_str=fecha_str
    truck_start_date_time_obtained=pd.to_datetime(fecha_next_str+" "+attention_start_time_obtained, format='%d-%m-%Y %H:%M:%S')
    Max_reach_date_time_at_destiny_p=Max_reach_time_at_destiny_p
    #print("inside check_destination_attention_v1")
    #print("max reach date time to reach at desyiny ",Max_reach_date_time_at_destiny_p)
    #     if plan=='10:30 PM':
    # #         if (attention_start_time_obtained=="22:00:00" or attention_start_time_obtained=="23:00:00"):
    #         truck_start_date_time_obtained=pd.to_datetime(fecha_str+attention_start_time_obtained, format='%Y-%m-%d%H:%M:%S')
    #     else:
    #         truck_start_date_time_obtained=pd.to_datetime(fecha_next_str+attention_start_time_obtained, format='%Y-%m-%d%H:%M:%S')
    if "-I" in destiny:
        destiny1 =destiny[:-2]
    else:
        destiny1 =destiny
    wait_hour=0    
    truck_reach_dest_date_time_obtained='NA'
    truck_reach_back_date_time_obtained='NA'
    #filter destiny attention hour table     
    destiny_attention_df_temp=destiny_attention_df[["COD","Attention_Start","Attention_End",day_of_week+"_attention"]]
    destiny_attention_df_temp=destiny_attention_df_temp[destiny_attention_df_temp.COD==destiny1]
    destiny_attention_df_temp['Attention_Start']=destiny_attention_df_temp['Attention_Start'].astype(str)

    #filter distance and time and find time to reach destiny 
    transfer_info =  distance_and_time[(distance_and_time['Origin_Code'] == origin) & (distance_and_time['Destiny_Code'] == destiny)]
    #print(transfer_info)
    one_way_time_in_min = int(transfer_info['time_one_way_min'].values[0])
    two_way_time_in_min = int(transfer_info['time_two_way_min'].values[0])*2
    loading_time_origin = int(transfer_info['Loading_time_origin_min'].values[0])
    unloading_time_at_dest = int(transfer_info['unloading_loading_time_in_dest_min'].values[0])
    break_time_min_dest =int(transfer_info['break_time_min'].values[0])
    reach_at_destination_date_time=truck_start_date_time_obtained +datetime.timedelta(minutes=loading_time_origin)+datetime.timedelta(minutes=one_way_time_in_min)
    print("reach_at_destination_date_time",reach_at_destination_date_time)
    print("Max_reach_date_time_at_destiny_p",Max_reach_date_time_at_destiny_p)
    #print(reach_at_destination_date_time>Max_reach_date_time_at_destiny_p)
    #print("will reach at destiny",reach_at_destination_date_time)
    reach_at_destination_time=reach_at_destination_date_time.time()
    reach_at_destination_time_ceil_down=reach_at_destination_time.hour
    reach_at_destination_time_ceil_down_time_format=datetime.time(reach_at_destination_time_ceil_down,0,0)
    destination_atention_hour_to_check=str(reach_at_destination_time_ceil_down_time_format)
    #print("destination attention hour  to be checked ",destination_atention_hour_to_check)
    #lets check destination attention hour 

    truck_sent =False
    attention_end_obtained="NA"
    #send_external_truck=False 
    duration_of_trip='NA'
    #Break time = 0

    #STEp0: DIrectly find destination attention hour 
    
    attention_available=destiny_attention_df_temp[(destiny_attention_df_temp['Attention_Start']==destination_atention_hour_to_check)][day_of_week+"_attention"].values[0]

    #Max_reach_date_time_at_destiny_p = max time at which trip can start from origin
    #STEP 1:
    if (reach_at_destination_date_time>Max_reach_date_time_at_destiny_p):
        #print("inside if 1")
        attention_end_obtained=destination_atention_hour_to_check
        truck_sent=True
        wait_hour=0
        truck_reach_dest_date_time_obtained=reach_at_destination_date_time+datetime.timedelta(hours=wait_hour)
        truck_reach_back_date_time_obtained=truck_reach_dest_date_time_obtained+\
        datetime.timedelta(minutes=one_way_time_in_min)+datetime.timedelta(minutes=unloading_time_at_dest)
        duration_of_trip=(truck_reach_back_date_time_obtained-truck_start_date_time_obtained).seconds/3600

    else :
        #print("inside else 1")
        if attention_available>0:
            #print("inside if 2")
            attention_end_obtained=destination_atention_hour_to_check
            truck_sent=True
            #print("truck can be send in own truck ")
        else :
            #print("inside else 2")
            #print("lets try to find attention hour after waiting ")
            attention_hour_list=get_attention_hour_list(destiny_attention_df,destiny1,destination_atention_hour_to_check)
            #print(attention_hour_list)
            #wait_hour_final=0
            #print(attention_hour_list)
            for attention_dest  in attention_hour_list:
                #### we have to send the truck right??
                if attention_dest==Max_reach_time_at_destiny_p:
                    #print("max time reached to reach at destiny  trip can not be planned")
                    break
                #print("currently checking for",attention_dest)
                attention_available=destiny_attention_df_temp[(destiny_attention_df_temp['Attention_Start']==attention_dest)][day_of_week+"_attention"].values[0]
                #print("Attention available ",attention_available)
                if attention_available>0:
                    attention_end_obtained=attention_dest
                    truck_sent=True
                    break
                wait_hour=wait_hour+1
        #wait_hour=wait_hour_final
        if truck_sent==True :
            truck_reach_dest_date_time_obtained=reach_at_destination_date_time+datetime.timedelta(hours=wait_hour)
            ##### Include Honduras remolque restrictions here
            #### Add the individual restrictions here
            truck_reach_back_date_time_obtained=truck_reach_dest_date_time_obtained+\
            datetime.timedelta(minutes=one_way_time_in_min)+datetime.timedelta(minutes=unloading_time_at_dest)+\
            datetime.timedelta(minutes=break_time_min_dest)
            duration_of_trip=(truck_reach_back_date_time_obtained - truck_start_date_time_obtained)/pd.Timedelta('1 hour')

    #     print("destination attention obtained",attention_end_obtained)
    #     print("can we send external truck",send_external_truck)
    #     print("can we send own truck",truck_sent)
    #     print("waiting time ",wait_hour)
    #print(truck_reach_back_date_time_obtained,truck_start_date_time_obtained)
    #    diff_time=
    #     print("total duration of trip ",(diff_time).seconds//3600)
    #     print("total duration of trip2 ",(diff_time.seconds//60)%60)
    #    print("total duration of trip2 ",(diff_time.seconds/3600))

    return truck_start_date_time_obtained,truck_reach_dest_date_time_obtained,truck_reach_back_date_time_obtained,attention_end_obtained,truck_sent,wait_hour,one_way_time_in_min,two_way_time_in_min,unloading_time_at_dest,duration_of_trip


In [268]:
# truck_start_date_time_obtained,truck_reach_dest_date_time_obtained,truck_reach_back_date_time_obtained,\
#             attention_end_obtained,truck_sent_dest,wait_hour,one_way_time_in_min,\
#             two_way_time_in_min,unloading_time_at_dest,duration_of_trip_hr=check_destination_attention_v1(destiny_attention_df,distance_and_time,origin,destiny,attention_start_time_obtained,Max_reach_date_time_at_destiny_p,fecha_str,day_of_week)

In [269]:
#Dist_plan1.columns

### Filler SKU

In [270]:
def check_update_filler_sku_V2(max_pallets,max_weight,result_df,filler_added,Filler_sku,filler_weight):
    #find how many pallets of filler SKU required 
    #but do not add filler SKU if total weight is crossing 40 ton 
    final_df_azua=result_df   
    req_cols=[x for x in final_df_azua.columns  if x.startswith("_")]
    final_df_required=final_df_azua[req_cols]
    sum_pallets=float(final_df_required.sum(axis=1))
    sum_total_weight=float(final_df_azua['Total_weight_in_kg'])
    filler_pallets_added=0
    filler_added=False
    #print("###initial ",sum_pallets,filler_pallets_added)
    while sum_pallets<max_pallets and sum_total_weight<max_weight:
        filler_pallets_added=filler_pallets_added+1
        sum_total_weight=sum_total_weight+filler_weight
        sum_pallets=sum_pallets+1
        ##print("###inside while ",sum_pallets,filler_pallets_added)
        filler_added=True
    pallets_required=filler_pallets_added
    #print("###",sum_pallets,pallets_required)
    final_df_azua["_"+str(Filler_sku)]=final_df_azua["_"+str(Filler_sku)]+pallets_required
    final_df_azua['Total_weight_in_kg']=sum_total_weight
    final_df_azua['total_pallets_sent']=sum_pallets
    return pallets_required,filler_added

In [271]:
def update_priority(result_df_row,df_priority_CND):
    #df_priority_CND['codigo']=df_priority_CND['codigo'].map('-{}'.format)
    #update priority
    final_df_x=result_df_row
    temp_dest=final_df_x['Destination'].values[0]
    temp_org=final_df_x['origin'].values[0]
    final_df_x=final_df_x.loc[:, (final_df_x != 0).any(axis=0)]
    req_col=[x for x in final_df_x.columns  if x.startswith("_")]
    final_df_x=final_df_x[req_col]
    dict_temp={}
    for k in final_df_x.columns:
        dict_temp[int(k.split("_")[1])]=final_df_x[k].values[0]
    df_priority_CND_required=df_priority_CND[(df_priority_CND.Destination==temp_dest)&((df_priority_CND.Origin==temp_org))]
    #print(df_priority_CND_required)
   # df_priority_CND_required['codigo']=df_priority_CND_required['codigo'].map('-{}'.format)
    for sku, pallet_sider in dict_temp.items():
        #print(sku, pallet_sider)
        if sku in df_priority_CND_required['codigo'].to_list():
            #print("sku updated ",sku)
            pallet_to_be_updated=df_priority_CND_required[df_priority_CND_required.codigo==sku]['Total_pallets'].values[0]
            #print("pallet_to_be_updated",pallet_to_be_updated)
            #print("pallet_sider",pallet_sider)
            #print(temp_dest)
            
            if pallet_to_be_updated<pallet_sider:
                pallet_to_be_updated=0
            else:
                pallet_to_be_updated=pallet_to_be_updated-pallet_sider
            #print("pallet_to_be_updated modified",pallet_to_be_updated)
            df_priority_CND.loc[(df_priority_CND.Destination==temp_dest) & (df_priority_CND.Origin==temp_org)&(df_priority_CND.codigo==sku),'Total_pallets']=pallet_to_be_updated
    return df_priority_CND

In [272]:
def update_origin_attention(origin_attention_df,origin,attention_start_hour_obtained,Day_of_week):
	#update origin attention 
    attention=origin_attention_df.loc[((origin_attention_df.CDD==origin) &(origin_attention_df.Attention_Start.astype(str)==attention_start_hour_obtained))][Day_of_week+"_attention"]
    attention=attention.values[0]

    if attention>0:
        origin_attention_df.loc[((origin_attention_df.CDD==origin) &(origin_attention_df.Attention_Start.astype(str)==attention_start_hour_obtained)),[Day_of_week+"_attention"]]=attention-1

def update_destination_attention(destiny_attention_df,destiny,attention_end_obtained,Day_of_week):
	#update destination attention 
    attention=destiny_attention_df.loc[((destiny_attention_df.CDD==destiny) &(destiny_attention_df.Attention_Start.astype(str)==attention_end_obtained))][Day_of_week+"_attention"]
    attention=attention.values[0]
    print(attention)

    if attention>0:
        destiny_attention_df.loc[((destiny_attention_df.CDD==destiny) &(destiny_attention_df.Attention_Start.astype(str)==attention_end_obtained)),[Day_of_week+"_attention"]]=attention-1


In [273]:
def rearrange_sider(current_day_fleet_siders_CND,ficha):
    #move the sider which is allocated to buttom 
    if ficha != 'NA':
        row=current_day_fleet_siders_CND.loc[current_day_fleet_siders_CND.Ficha==ficha]
        current_day_fleet_siders_CND=current_day_fleet_siders_CND[current_day_fleet_siders_CND.Ficha!=ficha]
        current_day_fleet_siders_CND=pd.concat([current_day_fleet_siders_CND,row])
        current_day_fleet_siders_CND=current_day_fleet_siders_CND.reset_index(drop=True)
    return current_day_fleet_siders_CND

In [274]:
### why arent we returning anything

def update_origin_attention(origin_attention_df,origin,attention_start_hour_obtained,Day_of_week):
	#update origin attention 
    attention=origin_attention_df.loc[((origin_attention_df.COD==origin) &(origin_attention_df.Attention_Start.astype(str)==attention_start_hour_obtained))][Day_of_week+"_attention"]
    attention=attention.values[0]

    if attention>0:
        origin_attention_df.loc[((origin_attention_df.COD==origin) &(origin_attention_df.Attention_Start.astype(str)==attention_start_hour_obtained)),[Day_of_week+"_attention"]]=attention-1

def update_destination_attention(destiny_attention_df,destiny,attention_end_obtained,Day_of_week):
	#update destination attention 
    attention=destiny_attention_df.loc[((destiny_attention_df.COD==destiny) &(destiny_attention_df.Attention_Start.astype(str)==attention_end_obtained))][Day_of_week+"_attention"]
    attention=attention.values[0]
    print(attention)

    if attention>0:
        destiny_attention_df.loc[((destiny_attention_df.COD==destiny) &(destiny_attention_df.Attention_Start.astype(str)==attention_end_obtained)),[Day_of_week+"_attention"]]=attention-1


In [275]:
def create_codigo_values(codigo_list,  total_pallets_list, weight_per_pallet_list):
    codigo_values = []
    #print(codigo_list)
    for i in range(len(codigo_list)):
        codigo_info = {
            "codigo": codigo_list[i],
            "Total_pallets": total_pallets_list[i],
            "kg_weight_per_pallet": weight_per_pallet_list[i]
        }
        codigo_values.append(codigo_info)
    return codigo_values

In [276]:
# codigo_values = create_codigo_values(vm['codigo'],vm[dest],vm['kg_weight_per_pallet'])
# x = {codigo["codigo"]: LpVariable(f"{codigo['codigo']}", lowBound=0, cat="Integer") for codigo in codigo_values}

In [277]:
def generate_recommendation_v2(vm,dest,max_pallets,max_weight):
    print("inside generate recommendation")
#     print("max_weight",max_weight)
#     print("max_pallet",max_pallets)
    import numpy as np 
    df = pd.DataFrame()
    i=0
    dict1 = {}
    vm['kg_weight_per_pallet']=vm['kg_weight_per_pallet'].astype(float)
    while (vm[dest].sum()>0):
        print("Start of while generate  recommendation ")
        if vm[dest].sum()>=max_pallets:
            print(i,vm[dest].sum())


            codigo_values = create_codigo_values(vm['codigo'],vm[dest],vm['kg_weight_per_pallet'])
            #print(codigo_values)

            from pulp import LpVariable, LpProblem, LpMinimize, lpSum, LpStatus ,LpMaximize,PULP_CBC_CMD
            problem = LpProblem("PalletShippingProblem", LpMaximize)
            max_pallets = max_pallets
            max_weight = max_weight

            x = {codigo["codigo"]: LpVariable(f"{codigo['codigo']}", lowBound=0, cat="Integer") for codigo in codigo_values}

            problem += lpSum(codigo["kg_weight_per_pallet"] * x[codigo["codigo"]] for codigo in codigo_values)
            # # # Set the constraints
            problem += lpSum(x.values()) <= max_pallets
            #problem += lpSum(codigo["kg_weight_per_pallet"] * x[codigo["codigo"]] for codigo in codigo_values) >= min_weight
            problem += lpSum(codigo["kg_weight_per_pallet"] * x[codigo["codigo"]] for codigo in codigo_values) <= max_weight
            #added y jitendra
            #problem += lpSum(codigo["kg_weight_per_pallet"] * x[codigo["codigo"]] for codigo in codigo_values) >=0 
            for codigo in codigo_values:
                problem +=x[codigo["codigo"]]<=codigo['Total_pallets']

			
            #problem.solve(PULP_CBC_CMD(msg=0))
            #print("prob solve started ")
            #print(problem)
            #problem.solve()
            problem.solve(PULP_CBC_CMD(msg=1, maxSeconds=0.5))
            #print("problem solve ended ")
            import numpy as np
            #print(problem)
            for var in problem.variables():
                dict1['Truck']=i
                dict1[var.name]=var.value()
                vm[dest] = np.where(vm["codigo"] == "-"+var.name.split("_")[1], vm[dest]-var.value(), vm[dest])
            dict1["Total_weight_in_kg"]=problem.objective.value()

            i=i+1

            df_temp = pd.DataFrame(dict1,index=[i])

            df_temp['Destination'] = dest

            df=df.append(df_temp)
        else:
            print("inside else ")
            print(i,vm[dest].sum())
            dict2={}
            dict2['Truck']=i
            
            total_kg=0
            #vm=vm.sort_values(by=['codigo'], ascending=True)
            #vm=vm.reset_index(drop=True )
            for j in range(vm.shape[0]):
                sku="_"+(vm.loc[j,'codigo']).split("-")[1]
                pallet=vm.loc[j,dest]
                sku_kg=pallet*vm.loc[j,'kg_weight_per_pallet']
                total_kg=total_kg+sku_kg
                dict2[sku]=pallet
                vm[dest] = np.where(vm["codigo"] == "-"+sku.split("_")[1], vm[dest]-pallet, vm[dest])

                
            
            dict2["Total_weight_in_kg"]=total_kg
            df_temp = pd.DataFrame(dict2,index=[i])
            df_temp['Destination'] = dest
            df=df.append(df_temp)

            

    return df


## Planning

In [278]:
#req_org_list=Dist_plan1['Origin'].unique()
req_org_list=Dist_plan1['org-priority'].unique()
req_dest_list=Dist_plan1['Destination'].unique()

#### Leave out DH14 because Roatan wil be in MVP2
req_dest_list =[x for x in req_dest_list if x != 'DH14']

In [279]:
Propia=list(current_day_fleet_siders[current_day_fleet_siders['Tipo de Vehículo']=='Propia']['Ficha'].unique())
Hibrid =list(current_day_fleet_siders[current_day_fleet_siders['Tipo de Vehículo']=='Hibrida']['Ficha'].unique())
Terceros=list(current_day_fleet_siders[current_day_fleet_siders['Tipo de Vehículo']=='Terceros']['Ficha'].unique())

propia_dest= ['DH00','DH01','DH03','DH15','DH11','DH10']
hibrid_dest= ['DH04','DH02','DH05','DH06','DH07','DH08','DH09','DH12','DH13']

In [280]:
Dist_plan1['Total_pallets'].sum()

5160.0

In [281]:
destiny = 'DH06'

In [282]:
if destiny in propia_dest:
    #### Sort in a way so that 1st propia and then hybrid and lastly thrid party fleet comes
    current_day_fleet_temp = current_day_fleet_siders[current_day_fleet_siders['Ficha'].isin(Propia)].append(current_day_fleet_siders[current_day_fleet_siders['Ficha'].isin(Hibrid)]).append(current_day_fleet_siders[current_day_fleet_siders['Ficha'].isin(Terceros)]).reset_index().drop(['index'],axis=1)
else:
    current_day_fleet_temp = current_day_fleet_siders[current_day_fleet_siders['Ficha'].isin(Hibrid)].append(current_day_fleet_siders[current_day_fleet_siders['Ficha'].isin(Terceros)]).reset_index().drop(['index'],axis=1)


In [283]:
current_day_fleet_siders[(current_day_fleet_siders.Ficha=='HN-MC78 HN-ML34')]

,Ficha,Tipo de Vehículo,Modelo,Capacidad KG,Capacidad Ton,Capacidad Pallets,Horas de utilidad,BREWERY,Tipo,Fleet_start_time,Fleet_start_date_time,Fleet_start_date_time_org,Max_reach_time_at_destiny,Ficha_Restriction_destiny,Ficha_Restriction_origin,Ficha_Restriction_sku


In [284]:
current_day_fleet_temp

,Ficha,Tipo de Vehículo,Modelo,Capacidad KG,Capacidad Ton,Capacidad Pallets,Horas de utilidad,BREWERY,Tipo,Fleet_start_time,Fleet_start_date_time,Fleet_start_date_time_org,Max_reach_time_at_destiny,Ficha_Restriction_destiny,Ficha_Restriction_origin,Ficha_Restriction_sku
0,HN-HC05 HN-MR78,Hibrida,HALL,25926.230656,41,28,24 Hr,all,Hibrida,00:00:01,2023-09-14 00:00:01,2023-09-14 00:00:01,2023-09-14 23:00:01,No restriction,No restriction,No restriction
1,HN-EC15 HN-MR99,Hibrida,TOM,25454.478515,41,28,24 Hr,all,Hibrida,00:00:01,2023-09-14 00:00:01,2023-09-14 00:00:01,2023-09-14 23:00:01,No restriction,No restriction,No restriction
2,HN-EC32 HN-MR91,Hibrida,EMTRATECA,25726.634261,41,28,24 Hr,all,Hibrida,00:00:01,2023-09-14 00:00:01,2023-09-14 00:00:01,2023-09-14 23:00:01,No restriction,No restriction,No restriction
3,HN-RC13 HN-VR24,Hibrida,RANSA,24752.301984,41,28,24 Hr,all,Hibrida,00:00:01,2023-09-14 00:00:01,2023-09-14 00:00:01,2023-09-14 23:00:01,No restriction,No restriction,No restriction
4,HN-TC10 HN-MR67,Hibrida,TOM,25880.871456,41,28,24 Hr,all,Hibrida,00:00:01,2023-09-14 00:00:01,2023-09-14 00:00:01,2023-09-14 23:00:01,No restriction,No restriction,No restriction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,HN-TC05 HN-ML03,Hibrida,TOM,25980.646989,41,28,24 Hr,all,Hibrida,00:00:01,2023-09-14 00:00:01,2023-09-14 00:00:01,2023-09-14 23:00:01,No restriction,No restriction,No restriction
153,HN-HC02 HN-HR05,Terceros,HALL,19282.660957,37,24,24 Hr,all,Terceros,00:00:01,2023-09-14 00:00:01,2023-09-14 00:00:01,2023-09-14 23:00:01,No restriction,No restriction,No restriction
154,HN-TC42 HN-TR11,Terceros,TOM,20008.425128,37,24,24 Hr,all,Terceros,00:00:01,2023-09-14 00:00:01,2023-09-14 00:00:01,2023-09-14 23:00:01,No restriction,No restriction,No restriction
155,HN-TC56 HN-TR02,Terceros,TOM,19618.319710,37,24,24 Hr,all,Terceros,00:00:01,2023-09-14 00:00:01,2023-09-14 00:00:01,2023-09-14 23:00:01,No restriction,No restriction,No restriction


In [285]:
#current_day_fleet_temp.append(current_day_fleet_siders_rest).reset_index().drop(['index'],axis=1)

In [286]:
Dist_plan1.to_excel('Data/Output/Distplan1_Jitendra.xlsx')

In [287]:
Dist_plan1.shape

(343, 9)

In [288]:
Dist_plan1=Dist_plan1.head(103)
#Dist_plan1.tail()

In [289]:
#dist_plan=dist_plan.reset_index(drop=True)
current_day_fleet_siders=current_day_fleet_siders.reset_index(drop=True)
pallets_max=0
weight_max=0
weight_max_kg=0
final_df=pd.DataFrame()
#origin='CND'
Day_of_week=day_of_week
last_assigned_ficha='NA'
iteration=0
#Dist_plan1['codigo']=Dist_plan1['codigo'].map('-{}'.format)
Filler_sku=Dist_plan1[(Dist_plan1['Prioridad']==1)&(Dist_plan1['Total_pallets']==Dist_plan1[Dist_plan1['Prioridad']==1]['Total_pallets'].max())].reset_index()['codigo'][0]
filler_weight=sku_description[sku_description['codigo']==Filler_sku].reset_index()['kg_weight_per_pallet'][0]
#commented by Jitendra 
#Dist_plan1=Dist_plan1.sort_values(['Prioridad','Total_pallets'],ascending=[True,False]).reset_index(drop=True)

while Dist_plan1.Total_pallets.sum()>0:
    if iteration==2:
        print("Break due  to max iteration")
        break 
    #restriction 
    
    
        
    for i in range(Dist_plan1.reset_index(drop=True).shape[0]):
        #print(Dist_plan1.loc[i])
        print("********************************************START***********")
        print("currently iterating distplan1 row number ",i)
        origin= Dist_plan1.loc[i,'Origin']
        destiny= Dist_plan1.loc[i,'Destination']
        sku= Dist_plan1.loc[i,'codigo']
        pallets=Dist_plan1.loc[i,'Total_pallets']
        #print("Pallets to sent from distplan1 ",pallets)
        #code added by Jitendra 
        print("Current combination",i,origin,destiny,sku,pallets)
        if pallets <=0:
            print("we have already covered this priority lets try next one ")
            continue 
        print("Distplan1")
        print(Dist_plan1[(Dist_plan1['Origin']==origin)&(Dist_plan1['Destination']==destiny)&(Dist_plan1['codigo']==sku)])
        #print(Dist_plan1)
        
        if pallets ==0.0:
            print("skipping destination sku Pallets needs to sent for this destination is 0",destiny,sku,i)
            continue 

        #if OrgPrBefMelt1[origin][destiny].sum()==0:
            #print(origin,destiny,"skipping dest")
        list1= Propia+Hibrid+Terceros
        list2 =Hibrid+Terceros
        if destiny in propia_dest:
            #### Sort in a way so that 1st propia and then hybrid and lastly thrid party fleet comes
            current_day_fleet_temp = current_day_fleet_siders[current_day_fleet_siders['Ficha'].isin(Propia)].append(current_day_fleet_siders[current_day_fleet_siders['Ficha'].isin(Hibrid)]).append(current_day_fleet_siders[current_day_fleet_siders['Ficha'].isin(Terceros)]).reset_index().drop(['index'],axis=1)
        else:
            current_day_fleet_temp = current_day_fleet_siders[current_day_fleet_siders['Ficha'].isin(Hibrid)].append(current_day_fleet_siders[current_day_fleet_siders['Ficha'].isin(Terceros)]).reset_index().drop(['index'],axis=1)

        ### Storing rest fleet in another df    
        current_day_fleet_siders_rest=current_day_fleet_siders[current_day_fleet_siders['Ficha'].isin(list(set(list(current_day_fleet_siders['Ficha'].unique())).difference(list(current_day_fleet_temp['Ficha'].unique()))))].reset_index().drop(['index'],axis=1)
        #current_day_fleet_temp.to_excel("data/current_day_fleet.xlsx")
        current_day_fleet_temp=current_day_fleet_temp[current_day_fleet_temp.Ficha!='HN-MC10  HN-ML09']
        #current_day_fleet_temp.to_excel("data/current_day_fleet.xlsx")


        
        for j in range(current_day_fleet_temp.shape[0]):
#             print("entering truck loop")
#             print("origin is",origin)
            print("j started",j)
#             print("destiny is",destiny)
#             print("pallets left in distribution plan",OrgPrBefMelt1[origin][destiny].sum())
            #Dist_plan1.to_excel('Data/Distplan1_now.xlsx')
            #print("ficha is ", ficha)
            ficha=current_day_fleet_temp.loc[j,'Ficha']
            print("ficha is ", ficha)
            #code added by Jitendra 
            if Dist_plan1.loc[i,'Total_pallets']<=0:
                print("The current Org dest SKU combination satisfied lets move to other ")
                break
            restr=current_day_fleet_temp.loc[j]['Ficha_Restriction_destiny']
            Type =current_day_fleet_temp.loc[j]['Tipo de Vehículo']
            if "-I" in restr:
                restr_1 =restr[:-2]
                dest1 =restr
            else:
                restr_1=restr
                dest1 =destiny
            #origin = org[:4]
            #destiny1 =dest1
            if (restr_1!='No restriction') and (destiny != restr_1):
                print("continue1")
                continue   
            if OrgPrBefMelt1[origin][destiny].sum()<1:
                print("Skipping this sider as sum of pallets found zero or less than 1",dest)
                break
            own_rented='company_owned'
            update_origin_attention_flag=False
            update_dest_attention_flag=False
            update_dist_plan=False
            update_sider=False
            filler_sku_added=False
            filler_added=False
            update_sider_start_time=False 
            can_send_in_own_sider=False 
            can_send_in_rented_sider=False
#             print("j is ",j)
#             print(current_day_fleet_temp.shape)
            ficha=current_day_fleet_temp.loc[j,'Ficha']
            fleet_start_date_time=current_day_fleet_temp.loc[j,'Fleet_start_date_time']
            fleet_start_date_time_org=current_day_fleet_temp.loc[j,'Fleet_start_date_time_org']
            fleet_start_time=str(fleet_start_date_time.hour).zfill(2)+":00:00"
            Max_reach_time_at_destiny=str(current_day_fleet_temp.loc[j,'Max_reach_time_at_destiny'].time())[0:5]+":"+"00"
            Max_reach_date_time_at_destiny=current_day_fleet_temp.loc[j,'Max_reach_time_at_destiny']

            print("currently_checking_for_ficha", ficha)
            if (fleet_start_date_time>Max_reach_date_time_at_destiny):
                print("This sider can not go  as start time at origin is > maximum time it should start from origin   ",fleet_start_date_time,Max_reach_date_time_at_destiny)
                continue

            #print("*****&&&",origin,fleet_start_time,Max_reach_time_at_destiny)
            #attention_start_time_obtained,truck_sent_origin=check_origin_attention_v1(origin_attention_df,origin,fleet_start_time,Max_reach_time_at_destiny,12,"Weekday") 
            attention_start_time_obtained,truck_sent_origin=check_origin_attention_v1\
            (origin_attention_df,origin,fleet_start_time,Max_reach_time_at_destiny,day_of_week,\
            attention_list_allowed,attention_dict_allowed) 
            if truck_sent_origin==False:
                print("This sider can not go  as we are not getting origin attention lets look for other sider  ",ficha)
                continue
            #print("can start",attention_start_time_obtained)

            #check destination attention 
            truck_start_date_time_obtained,truck_reach_dest_date_time_obtained,truck_reach_back_date_time_obtained,\
            attention_end_obtained,truck_sent_dest,wait_hour,one_way_time_in_min,\
            two_way_time_in_min,unloading_time_at_dest,duration_of_trip_hr=check_destination_attention_v1(destiny_attention_df,distance_and_time,origin,dest1,attention_start_time_obtained,Max_reach_date_time_at_destiny,fecha_str,day_of_week)

            if truck_sent_dest==False:
                print("This sider can not go to destination even after waiting  ",ficha)
                continue
#             print("truck_sent_dest",truck_sent_dest)
#             print("can_send_in_own_sider",can_send_in_own_sider)
#             print("can_send_in_rented_sider",can_send_in_rented_sider)
            if truck_sent_dest==True:
                #print("company owned sider will be used ")
                can_send_in_own_sider=True
                update_origin_attention_flag=True
                update_dest_attention_flag=True
                update_sider_start_time=True

            if can_send_in_own_sider==True:
                pallets_max=current_day_fleet_temp.loc[j,'Capacidad Pallets']
                weight_max=current_day_fleet_temp.loc[j,'Capacidad Ton']
                weight_max_kg=current_day_fleet_temp.loc[j,'Capacidad KG']

            max_pallets=pallets_max
            max_weight=weight_max_kg
#             print("can_send_in_own_sider",can_send_in_own_sider)
#             print("can_send_in_rented_sider",can_send_in_rented_sider)
#             print("MAximum",max_pallets, max_weight)

            dest=destiny

            OrgPrBefMelt1[origin]['codigo']=OrgPrBefMelt1[origin]['codigo'].map('-{}'.format)
            vm = OrgPrBefMelt1[origin][['codigo',dest,'kg_weight_per_pallet']].copy()
            #print(vm.shape)
            vm=vm.reset_index(drop=True)
            #vm.to_excel('data/current_vm.xlsx')
            #print(vm.head(2))
            from pulp import LpVariable, LpProblem, LpMinimize, lpSum, LpStatus ,LpMaximize
            
            df1=generate_recommendation_v2(vm,dest,max_pallets,max_weight)
            df1=df1.sort_values(by ="_"+str(sku),ascending = False).head(n=1)
            result_df=df1.head(1)
            result_df['origin']=origin
            result_df['Fleet']=ficha
            result_df['Type']=Type
            result_df['Rest-Individual']=dest1
            result_df['Rest-Individual'].fillna('no rest')
            #print(result_df)

            Dist_plan1=update_priority(result_df,Dist_plan1)
            Dist_plan1=Dist_plan1.reset_index().drop(['index'],axis=1)
#             Dist_plan1=Dist_plan1.sort_values(['Prioridad'],ascending=[True])
            OrgPr1={}
            for m in list(Dist_plan1['Origin'].unique()):
                #print(i)
                temp = Dist_plan1[Dist_plan1['Origin']==m]
                OrgPr1[m]=temp

            OrgPrBefMelt1={}
            for n in list(OrgPr1.keys()):
                #print(i) 
                temp1=OrgPr1[n].pivot(index='codigo', columns='Destination', values='Total_pallets').fillna(0).reset_index().rename_axis(None, axis=1)
                OrgPrBefMelt1[n]=temp1
                #### Ensuring all datasets(all or-dest-priority comb) have the exhaustive list of destinities-total 15.
                diff_col=list(set(All_destiny).difference(list(OrgPrBefMelt1[n].columns)))
                for l in diff_col:
                    OrgPrBefMelt1[n][l]=0

            # Ensuring all datasets(all or-dest-priority comb) have the exhaustive list of SKUs.
            for p in OrgPrBefMelt1.keys():
                OrgPrBefMelt1[p]=OrgPrBefMelt1[p].append(pd.DataFrame(list(set(All_sku).difference(list(OrgPrBefMelt1[p]['codigo'].unique()))), columns=['codigo'])).fillna(0).reset_index().drop(['index'],axis=1)
                OrgPrBefMelt1[p]=OrgPrBefMelt1[p].merge(sku_description[['codigo','kg_weight_per_pallet']])

            total_pallets_sent=result_df[[x for x in result_df.columns  if x.startswith("_")]].sum(axis=1)

            result_df["total_pallets_sent"]=total_pallets_sent
            #result_df["own_rented"]=own_rented
            result_df["Original_sider_start_date_time"]=fleet_start_date_time_org
            result_df["Origin_attention_obtained_sider"]=truck_start_date_time_obtained
            result_df["one_way_time_in_min"]=one_way_time_in_min
            #result_df["two_way_time_in_min"]=two_way_time_in_min
            result_df["unloading_time_at_dest"]=unloading_time_at_dest

            result_df["sider_reach_dest_date_time_obtained"]=truck_reach_dest_date_time_obtained
            result_df["Sider_wait_time_at_destiny"]=wait_hour
            result_df["sider_reach_back_date_time_obtained"]=truck_reach_back_date_time_obtained
            result_df["total_trip_time_hr"]=duration_of_trip_hr

            pallets_required_filler,filler_added= check_update_filler_sku_V2(max_pallets,max_weight,result_df,filler_added,Filler_sku,filler_weight)

            result_df["filler_added"]=filler_added
            result_df["filler_added_pallets"]=pallets_required_filler
            result_df["priority"]="Priority"
            #result_df["ficha_sider"]=ficha

            #UPDATE ORIGIN ATTENTION
            if update_origin_attention_flag==True:

                #print(origin,attention_start_time_obtained,Day_of_week)
                update_origin_attention(origin_attention_df,origin,attention_start_time_obtained,Day_of_week)

            #UPDATE DESTINATION ATTENTION
            if update_dest_attention_flag==True:
                update_destination_attention(destiny_attention_df,destiny,attention_end_obtained,Day_of_week)

            #UPDATE SIDER START DATE TIME AS ORIGIN ATTENTION OBTAINED 
            if update_sider_start_time==True :
                current_day_fleet_temp.loc[current_day_fleet_temp.Ficha==ficha,'Fleet_start_date_time']=truck_reach_back_date_time_obtained

            final_df=pd.concat([final_df,result_df])
            last_assigned_ficha=ficha

            current_day_fleet_siders=current_day_fleet_temp.append(current_day_fleet_siders_rest).reset_index().drop(['index'],axis=1)
            current_day_fleet_siders=rearrange_sider(current_day_fleet_siders,last_assigned_ficha)
            #code added by Jitendra 
            print("$$$$$",Dist_plan1.loc[i,'Total_pallets'] )
            print(i,j)
    iteration=iteration+1


********************************************START***********
currently iterating distplan1 row number  0
Current combination 0 BH01 DH00 13917 132.0
Distplan1
  Origin Destination codigo  Prioridad  Total_boxes  kg_weight_per_pallet  \
0   BH01        DH00  13917          1         6468             847.65005   

   package_per_pallet  Total_pallets org-priority  
0                49.0          132.0       BH01-1  
j started 0
ficha is  HN-1333  HN-1064
currently_checking_for_ficha HN-1333  HN-1064
reach_at_destination_date_time 2023-09-14 03:00:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
inside generate recommendation
Start of while generate  recommendation 
0 560.0
Start of while generate  recommendation 
1 536.0
Start of while generate  recommendation 
2 512.0
Start of while generate  recommendation 
3 488.0
Start of while generate  recommendation 
4 464.0
Start of while generate  recommendation 
5 440.0
Start of while generate  recommendation 
6 416.0
Start of while gene

Start of while generate  recommendation 
inside else 
2 9.0
3
$$$$$ 39.0
1 19
j started 20
ficha is  HN-MC03  HN-VR33
continue1
j started 21
ficha is  HN-MC62  HN-VR71
continue1
j started 22
ficha is  HN-MC72  HN-ML06
continue1
j started 23
ficha is  HN-MC16  HN-MR24
continue1
j started 24
ficha is  HN-MC04  HN-MR18
continue1
j started 25
ficha is  HN-MC58  HN-MR32
continue1
j started 26
ficha is  HN-MC21  HN-VR91
continue1
j started 27
ficha is  HN-1333  HN-1684
currently_checking_for_ficha HN-1333  HN-1684
reach_at_destination_date_time 2023-09-14 04:00:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
inside generate recommendation
Start of while generate  recommendation 
0 39.0
Start of while generate  recommendation 
inside else 
1 15.0
1
$$$$$ 15.0
1 27
j started 28
ficha is  HN-1333  HN-1685
currently_checking_for_ficha HN-1333  HN-1685
reach_at_destination_date_time 2023-09-14 05:00:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
inside generate recommendation
Sta

Start of while generate  recommendation 
1 53.0
Start of while generate  recommendation 
inside else 
2 25.0
1
$$$$$ 12.0
9 0
j started 1
ficha is  HN-EC32  HN-MR91
currently_checking_for_ficha HN-EC32  HN-MR91
reach_at_destination_date_time 2023-09-14 13:30:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
inside generate recommendation
Start of while generate  recommendation 
0 53.0
Start of while generate  recommendation 
inside else 
1 25.0
1
$$$$$ 0.0
9 1
j started 2
ficha is  HN-RC13  HN-VR24
The current Org dest SKU combination satisfied lets move to other 
********************************************START***********
currently iterating distplan1 row number  10
Current combination 10 BH01 DH04 13917 5.0
Distplan1
   Origin Destination codigo  Prioridad  Total_boxes  kg_weight_per_pallet  \
10   BH01        DH04  13917          1         1029             847.65005   

    package_per_pallet  Total_pallets org-priority  
10                49.0            5.0       BH01-1  
j

0
$$$$$ 0.0
25 0
j started 1
ficha is  HN-EC19  HN-VR42
The current Org dest SKU combination satisfied lets move to other 
********************************************START***********
currently iterating distplan1 row number  26
Current combination 26 BH01 DH09 18410 0.0
we have already covered this priority lets try next one 
********************************************START***********
currently iterating distplan1 row number  27
Current combination 27 BH01 DH10 13910 4.0
Distplan1
   Origin Destination codigo  Prioridad  Total_boxes  kg_weight_per_pallet  \
27   BH01        DH10  13910          1          432           1010.602976   

    package_per_pallet  Total_pallets org-priority  
27               108.0            4.0       BH01-1  
j started 0
ficha is  HN-1333  HN-1064
continue1
j started 1
ficha is  HN-MC70  HN-MR27
continue1
j started 2
ficha is  HN-MC02  HN-ML37
continue1
j started 3
ficha is  HN-VC19-2  HN-VR80
currently_checking_for_ficha HN-VC19-2  HN-VR80
reach_at_dest

1
$$$$$ 0.0
37 0
j started 1
ficha is  HN-XC10  HN-VR50
The current Org dest SKU combination satisfied lets move to other 
********************************************START***********
currently iterating distplan1 row number  38
Current combination 38 BH01 DH15 13915 2.0
Distplan1
   Origin Destination codigo  Prioridad  Total_boxes  kg_weight_per_pallet  \
38   BH01        DH15  13915          1           72            665.238027   

    package_per_pallet  Total_pallets org-priority  
38                36.0            2.0       BH01-1  
j started 0
ficha is  HN-1333  HN-1064
continue1
j started 1
ficha is  HN-MC70  HN-MR27
currently_checking_for_ficha HN-MC70  HN-MR27
reach_at_destination_date_time 2023-09-15 04:30:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
inside generate recommendation
Start of while generate  recommendation 
0 43.0
Start of while generate  recommendation 
inside else 
1 15.0
1
$$$$$ 0.0
38 1
j started 2
ficha is  HN-MC02  HN-ML37
The current Org dest 

...

Start of while generate  recommendation 
inside else 
1 1.0
1
$$$$$ 0.0
76 0
j started 1
ficha is  HN-TC30  HN-MR61
The current Org dest SKU combination satisfied lets move to other 
********************************************START***********
currently iterating distplan1 row number  77
Current combination 77 SH01 DH06 13977 1.0
Distplan1
   Origin Destination codigo  Prioridad  Total_boxes  kg_weight_per_pallet  \
77   SH01        DH06  13977          1          360            783.625539   

    package_per_pallet  Total_pallets org-priority  
77               120.0            1.0       SH01-1  
j started 0
ficha is  HN-TC30  HN-MR61
currently_checking_for_ficha HN-TC30  HN-MR61
reach_at_destination_date_time 2023-09-14 08:30:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
This sider can not go to destination even after waiting   HN-TC30  HN-MR61
j started 1
ficha is  HN-XC18  HN-VR54
currently_checking_for_ficha HN-XC18  HN-VR54
reach_at_destination_date_time 2023-09-14 08:3

reach_at_destination_date_time 2023-09-14 08:30:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
This sider can not go to destination even after waiting   HN-EC41  HN-MR92
j started 32
ficha is  HN-YC02  HN-MR34
currently_checking_for_ficha HN-YC02  HN-MR34
reach_at_destination_date_time 2023-09-14 08:30:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
This sider can not go to destination even after waiting   HN-YC02  HN-MR34
j started 33
ficha is  HN-RC17  HN-VR77
currently_checking_for_ficha HN-RC17  HN-VR77
reach_at_destination_date_time 2023-09-14 08:30:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
This sider can not go to destination even after waiting   HN-RC17  HN-VR77
j started 34
ficha is  HN-NC09  HN-MR73
currently_checking_for_ficha HN-NC09  HN-MR73
reach_at_destination_date_time 2023-09-14 08:30:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
This sider can not go to destination even after waiting   HN-NC09  HN-MR73
j started 35
ficha is  HN-

This sider can not go to destination even after waiting   HN-EC11  HN-MR60
j started 70
ficha is  HN-YC01  HN-VR26
currently_checking_for_ficha HN-YC01  HN-VR26
reach_at_destination_date_time 2023-09-14 08:30:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
This sider can not go to destination even after waiting   HN-YC01  HN-VR26
j started 71
ficha is  HN-EC37  HN-MR52
currently_checking_for_ficha HN-EC37  HN-MR52
reach_at_destination_date_time 2023-09-14 08:30:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
This sider can not go to destination even after waiting   HN-EC37  HN-MR52
j started 72
ficha is  HN-TC32  HN-MR85
currently_checking_for_ficha HN-TC32  HN-MR85
reach_at_destination_date_time 2023-09-14 08:30:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
This sider can not go to destination even after waiting   HN-TC32  HN-MR85
j started 73
ficha is  HN-TC40  HN-MR57
currently_checking_for_ficha HN-TC40  HN-MR57
reach_at_destination_date_time 2023-09-14 08

This sider can not go to destination even after waiting   HN-TC39  HN-MR36
j started 105
ficha is  HN-EC20  HN-MR48
currently_checking_for_ficha HN-EC20  HN-MR48
reach_at_destination_date_time 2023-09-14 08:30:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
This sider can not go to destination even after waiting   HN-EC20  HN-MR48
j started 106
ficha is  HN-EC24  HN-MR93
currently_checking_for_ficha HN-EC24  HN-MR93
reach_at_destination_date_time 2023-09-14 08:30:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
This sider can not go to destination even after waiting   HN-EC24  HN-MR93
j started 107
ficha is  HN-NC15  HN-VR04
currently_checking_for_ficha HN-NC15  HN-VR04
reach_at_destination_date_time 2023-09-14 08:30:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
This sider can not go to destination even after waiting   HN-NC15  HN-VR04
j started 108
ficha is  HN-TC17  HN-MR53
currently_checking_for_ficha HN-TC17  HN-MR53
reach_at_destination_date_time 2023-09-1

Start of while generate  recommendation 
1 93.0
Start of while generate  recommendation 
2 66.0
Start of while generate  recommendation 
3 41.0
Start of while generate  recommendation 
inside else 
4 15.0
1
$$$$$ 5.0
92 0
j started 1
ficha is  HN-XC18  HN-VR54
currently_checking_for_ficha HN-XC18  HN-VR54
reach_at_destination_date_time 2023-09-14 18:30:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
inside generate recommendation
Start of while generate  recommendation 
0 93.0
Start of while generate  recommendation 
1 67.0
Start of while generate  recommendation 
2 41.0
Start of while generate  recommendation 
inside else 
3 17.0
1
$$$$$ 1.0
92 1
j started 2
ficha is  HN-RC11  HN-VR52
currently_checking_for_ficha HN-RC11  HN-VR52
reach_at_destination_date_time 2023-09-14 18:30:00
Max_reach_date_time_at_destiny_p 2023-09-14 23:00:01
inside generate recommendation
Start of while generate  recommendation 
0 67.0
Start of while generate  recommendation 
1 43.0
Start of while gener

In [290]:
Dist_plan1[Dist_plan1.Total_pallets!=0]


,Origin,Destination,codigo,Prioridad,Total_boxes,kg_weight_per_pallet,package_per_pallet,Total_pallets,org-priority


In [291]:
current_day_fleet_siders.head()

,Ficha,Tipo de Vehículo,Modelo,Capacidad KG,Capacidad Ton,Capacidad Pallets,Horas de utilidad,BREWERY,Tipo,Fleet_start_time,Fleet_start_date_time,Fleet_start_date_time_org,Max_reach_time_at_destiny,Ficha_Restriction_destiny,Ficha_Restriction_origin,Ficha_Restriction_sku
0,HN-TC30 HN-MR61,Hibrida,TOM,25926.215949,41,28,24 Hr,all,Hibrida,00:00:01,2023-09-16 00:30:00,2023-09-14 00:00:01,2023-09-14 23:00:01,No restriction,No restriction,No restriction
1,HN-XC18 HN-VR54,Hibrida,EXA,25125.996320,41,28,24 Hr,all,Hibrida,00:00:01,2023-09-16 01:30:00,2023-09-14 00:00:01,2023-09-14 23:00:01,No restriction,No restriction,No restriction
2,HN-TC07 HN-MR51,Hibrida,TOM,25817.368576,41,28,24 Hr,all,Hibrida,00:00:01,2023-09-14 00:00:01,2023-09-14 00:00:01,2023-09-14 23:00:01,No restriction,No restriction,No restriction
3,HN-ZC01 HN-VR06,Hibrida,ZAVALA,25278.501280,41,28,24 Hr,all,Hibrida,00:00:01,2023-09-14 00:00:01,2023-09-14 00:00:01,2023-09-14 23:00:01,No restriction,No restriction,No restriction
4,HN-NC08 HN-MR74,Hibrida,NUÑEZ,25273.058176,41,28,24 Hr,all,Hibrida,00:00:01,2023-09-14 00:00:01,2023-09-14 00:00:01,2023-09-14 23:00:01,No restriction,No restriction,No restriction


In [292]:
Dist_plan1.loc[0,'Total_pallets']

0.0

In [293]:
Dist_plan2[(Dist_plan2['Origin']=='SH01')&(Dist_plan2['Destination']=='DH13')&(Dist_plan2['codigo']==14623)]

,Origin,Destination,codigo,Prioridad,Total_boxes,kg_weight_per_pallet,package_per_pallet,Total_pallets,org-priority
152,SH01,DH13,14623,1,1392,953.994694,48.0,29.0,SH01-1


In [294]:
Dist_plan1.to_excel('Data/Output/Dist_plan1_remaining_Jitendra.xlsx')

In [295]:
#Dist_plan2.to_excel('Data/Dist_plan1.xlsx')

In [296]:
Dist_plan1['Prioridad'][Dist_plan1['Total_pallets']>0].value_counts()

Series([], Name: Prioridad, dtype: int64)

In [297]:
Dist_plan2[(Dist_plan2['Origin']=='SH01')&(Dist_plan2['Destination']=='DH06')]['Total_pallets'].sum()

168.0

In [298]:
final_df

,Truck,_13908,_13910,_13911,_13912,_13915,_13917,_13918,_13919,_13925,_13927,_13930,_13956,_13957,_13958,_13961,_13964,_13965,_13967,_13973,_13975,_13977,_13986,_13990,_13991,_14014,_14018,_14030,_14552,_14619,_14620,_14621,_14622,_14623,_15760,_15762,_15768,_16104,_16139,_16682,_17100,_17365,_17739,_18410,Total_weight_in_kg,Destination,origin,Fleet,Type,Rest-Individual,total_pallets_sent,Original_sider_start_date_time,Origin_attention_obtained_sider,one_way_time_in_min,unloading_time_at_dest,sider_reach_dest_date_time_obtained,Sider_wait_time_at_destiny,sider_reach_back_date_time_obtained,total_trip_time_hr,filler_added,filler_added_pallets,priority
13,12,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20343.601200,DH00,BH01,HN-1333 HN-1064,Propia,DH00,24.0,2023-09-14 00:00:01,2023-09-14 01:00:00,30,30,2023-09-14 03:00:00,0,2023-09-14 04:30:00,3.5,False,0,Priority
1,0,0.0,0.0,0.0,0.0,0.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23734.201400,DH00,BH01,HN-MC26 HN-VR79,Propia,DH00,28.0,2023-09-14 00:00:01,2023-09-14 01:00:00,30,30,2023-09-14 03:00:00,0,2023-09-14 04:30:00,3.5,False,0,Priority
1,0,0.0,0.0,0.0,0.0,0.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23734.201400,DH00,BH01,HN-VC19-2 HN-VR80,Propia,DH00,28.0,2023-09-14 00:00:01,2023-09-14 01:00:00,30,30,2023-09-14 03:00:00,0,2023-09-14 04:30:00,3.5,False,0,Priority
1,0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25429.501500,DH00,BH01,HN-MC78 HN-ML34,Propia,DH00,30.0,2023-09-14 00:00:01,2023-09-14 02:00:00,30,30,2023-09-14 04:00:00,0,2023-09-14 05:30:00,3.5,False,0,Priority
1,0,0.0,0.0,0.0,0.0,0.0,22.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19977.470809,DH00,BH01,HN-1333 HN-1065,Propia,DH00,24.0,2023-09-14 00:00:01,2023-09-14 02:00:00,30,30,2023-09-14 04:00:00,0,2023-09-14 05:30:00,3.5,False,0,Priority
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,1.0,4.0,0.0,0.0,0.0,1.0,15.0,0.0,0.0,27004.826516,DH08,SH01,HN-TC30 HN-MR61,Hibrida,DH08,28.0,2023-09-14 00:00:01,2023-09-14 05:00:00,660,60,2023-09-15 00:30:00,6,2023-09-16 00:30:00,43.5,True,1,Priority
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,26204.599510,DH08,SH01,HN-XC18 HN-VR54,Hibrida,DH08,27.0,2023-09-14 00:00:01,2023-09-14 05:00:00,660,60,2023-09-15 01:30:00,7,2023-09-16 01:30:00,44.5,True,1,Priority
2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,25726.105309,DH08,SH01,HN-RC11 HN-VR52,Hibrida,DH08,25.0,2023-09-14 00:00:01,2023-09-14 05:00:00,660,60,2023-09-15 02:30:00,8,2023-09-16 02:30:00,45.5,True,1,Priority
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,7.0,0.0,0.0,27192.114653,DH08,SH01,HN-XC14 HN-VR47,Hibrida,DH08,27.0,2023-09-14 00:00:01,2023-09-14 05:00:00,660,60,2023-09-15 03:30:00,9,2023-09-16 03:30:00,46.5,True,1,Priority


In [299]:
final_df.to_excel('Data/Output/Result_Honduras_14thSep_final_V3_Jitendra.xlsx')

In [300]:
final_df

,Truck,_13908,_13910,_13911,_13912,_13915,_13917,_13918,_13919,_13925,_13927,_13930,_13956,_13957,_13958,_13961,_13964,_13965,_13967,_13973,_13975,_13977,_13986,_13990,_13991,_14014,_14018,_14030,_14552,_14619,_14620,_14621,_14622,_14623,_15760,_15762,_15768,_16104,_16139,_16682,_17100,_17365,_17739,_18410,Total_weight_in_kg,Destination,origin,Fleet,Type,Rest-Individual,total_pallets_sent,Original_sider_start_date_time,Origin_attention_obtained_sider,one_way_time_in_min,unloading_time_at_dest,sider_reach_dest_date_time_obtained,Sider_wait_time_at_destiny,sider_reach_back_date_time_obtained,total_trip_time_hr,filler_added,filler_added_pallets,priority
13,12,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20343.601200,DH00,BH01,HN-1333 HN-1064,Propia,DH00,24.0,2023-09-14 00:00:01,2023-09-14 01:00:00,30,30,2023-09-14 03:00:00,0,2023-09-14 04:30:00,3.5,False,0,Priority
1,0,0.0,0.0,0.0,0.0,0.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23734.201400,DH00,BH01,HN-MC26 HN-VR79,Propia,DH00,28.0,2023-09-14 00:00:01,2023-09-14 01:00:00,30,30,2023-09-14 03:00:00,0,2023-09-14 04:30:00,3.5,False,0,Priority
1,0,0.0,0.0,0.0,0.0,0.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23734.201400,DH00,BH01,HN-VC19-2 HN-VR80,Propia,DH00,28.0,2023-09-14 00:00:01,2023-09-14 01:00:00,30,30,2023-09-14 03:00:00,0,2023-09-14 04:30:00,3.5,False,0,Priority
1,0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25429.501500,DH00,BH01,HN-MC78 HN-ML34,Propia,DH00,30.0,2023-09-14 00:00:01,2023-09-14 02:00:00,30,30,2023-09-14 04:00:00,0,2023-09-14 05:30:00,3.5,False,0,Priority
1,0,0.0,0.0,0.0,0.0,0.0,22.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19977.470809,DH00,BH01,HN-1333 HN-1065,Propia,DH00,24.0,2023-09-14 00:00:01,2023-09-14 02:00:00,30,30,2023-09-14 04:00:00,0,2023-09-14 05:30:00,3.5,False,0,Priority
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,1.0,4.0,0.0,0.0,0.0,1.0,15.0,0.0,0.0,27004.826516,DH08,SH01,HN-TC30 HN-MR61,Hibrida,DH08,28.0,2023-09-14 00:00:01,2023-09-14 05:00:00,660,60,2023-09-15 00:30:00,6,2023-09-16 00:30:00,43.5,True,1,Priority
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,26204.599510,DH08,SH01,HN-XC18 HN-VR54,Hibrida,DH08,27.0,2023-09-14 00:00:01,2023-09-14 05:00:00,660,60,2023-09-15 01:30:00,7,2023-09-16 01:30:00,44.5,True,1,Priority
2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,25726.105309,DH08,SH01,HN-RC11 HN-VR52,Hibrida,DH08,25.0,2023-09-14 00:00:01,2023-09-14 05:00:00,660,60,2023-09-15 02:30:00,8,2023-09-16 02:30:00,45.5,True,1,Priority
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,7.0,0.0,0.0,27192.114653,DH08,SH01,HN-XC14 HN-VR47,Hibrida,DH08,27.0,2023-09-14 00:00:01,2023-09-14 05:00:00,660,60,2023-09-15 03:30:00,9,2023-09-16 03:30:00,46.5,True,1,Priority


In [ ]:
# import datetime
# from datetime import timedelta
# fecha_next_str=fecha_str
# truck_start_date_time_obtained=pd.to_datetime(fecha_next_str+" "+attention_start_time_obtained, format='%d-%m-%Y %H:%M:%S')
# Max_reach_date_time_at_destiny_p=Max_reach_date_time_at_destiny
# #print("inside check_destination_attention_v1")
# #print("max reach date time to reach at desyiny ",Max_reach_date_time_at_destiny_p)
# #     if plan=='10:30 PM':
# # #         if (attention_start_time_obtained=="22:00:00" or attention_start_time_obtained=="23:00:00"):
# #         truck_start_date_time_obtained=pd.to_datetime(fecha_str+attention_start_time_obtained, format='%Y-%m-%d%H:%M:%S')
# #     else:
# #         truck_start_date_time_obtained=pd.to_datetime(fecha_next_str+attention_start_time_obtained, format='%Y-%m-%d%H:%M:%S')
# if "-I" in destiny:
#     destiny1 =destiny[:-2]
# else:
#     destiny1 =destiny
# wait_hour=0    
# truck_reach_dest_date_time_obtained='NA'
# truck_reach_back_date_time_obtained='NA'
# #filter destiny attention hour table     
# destiny_attention_df_temp=destiny_attention_df[["COD","Attention_Start","Attention_End",day_of_week+"_attention"]]
# destiny_attention_df_temp=destiny_attention_df_temp[destiny_attention_df_temp.COD==destiny1]
# destiny_attention_df_temp['Attention_Start']=destiny_attention_df_temp['Attention_Start'].astype(str)

# #filter distance and time and find time to reach destiny 
# transfer_info =  distance_and_time[(distance_and_time['Origin_Code'] == origin) & (distance_and_time['Destiny_Code'] == destiny)]
# #print(transfer_info)
# one_way_time_in_min = int(transfer_info['time_one_way_min'].values[0])
# two_way_time_in_min = int(transfer_info['time_two_way_min'].values[0])*2
# loading_time_origin = int(transfer_info['Loading_time_origin_min'].values[0])
# unloading_time_at_dest = int(transfer_info['unloading_loading_time_in_dest_min'].values[0])
# break_time_min_dest =int(transfer_info['break_time_min'].values[0])
# reach_at_destination_date_time=truck_start_date_time_obtained +datetime.timedelta(minutes=loading_time_origin)+datetime.timedelta(minutes=one_way_time_in_min)
# print("reach_at_destination_date_time",reach_at_destination_date_time)
# print("Max_reach_date_time_at_destiny_p",Max_reach_date_time_at_destiny_p)
# #print(reach_at_destination_date_time>Max_reach_date_time_at_destiny_p)
# #print("will reach at destiny",reach_at_destination_date_time)
# reach_at_destination_time=reach_at_destination_date_time.time()
# reach_at_destination_time_ceil_down=reach_at_destination_time.hour
# reach_at_destination_time_ceil_down_time_format=datetime.time(reach_at_destination_time_ceil_down,0,0)
# destination_atention_hour_to_check=str(reach_at_destination_time_ceil_down_time_format)
# #print("destination attention hour  to be checked ",destination_atention_hour_to_check)
# #lets check destination attention hour 

# truck_sent =False
# attention_end_obtained="NA"
# #send_external_truck=False 
# duration_of_trip='NA'
# #Break time = 0

# #STEp0: DIrectly find destination attention hour 

# attention_available=destiny_attention_df_temp[(destiny_attention_df_temp['Attention_Start']==destination_atention_hour_to_check)][day_of_week+"_attention"].values[0]

# #Max_reach_date_time_at_destiny_p = max time at which trip can start from origin
# #STEP 1:
# if (reach_at_destination_date_time>Max_reach_date_time_at_destiny_p):
#     attention_end_obtained=destination_atention_hour_to_check
#     truck_sent=True
#     wait_hour=0
#     truck_reach_dest_date_time_obtained=reach_at_destination_date_time+datetime.timedelta(hours=wait_hour)
#     truck_reach_back_date_time_obtained=truck_reach_dest_date_time_obtained+\
#     datetime.timedelta(minutes=one_way_time_in_min)+datetime.timedelta(minutes=unloading_time_at_dest)
#     duration_of_trip=(truck_reach_back_date_time_obtained-truck_start_date_time_obtained).seconds/3600

# else :
#     if attention_available>0:
#         attention_end_obtained=destination_atention_hour_to_check
#         truck_sent=True
#         #print("truck can be send in own truck ")
#     else :
#         #print("lets try to find attention hour after waiting ")
#         attention_hour_list=get_attention_hour_list(destiny_attention_df,destiny,destination_atention_hour_to_check)
#         #wait_hour_final=0
#         #print(attention_hour_list)
#         for attention_dest  in attention_hour_list:
#             #### we have to send the truck right??
#             if attention_dest==Max_reach_time_at_destiny_p:
#                 print("max time reached to reach at destiny  trip can not be planned")
#                 break
#             #print("currently checking for",attention_dest)
#             attention_available=destiny_attention_df_temp[(destiny_attention_df_temp['Attention_Start']==attention_dest)][day_of_week+"_attention"].values[0]
#             #print("Attention available ",attention_available)
#             if attention_available>0:
#                 attention_end_obtained=attention_dest
#                 truck_sent=True
#                 break
#             wait_hour=wait_hour+1
#     #wait_hour=wait_hour_final
#     if truck_sent==True :
#         truck_reach_dest_date_time_obtained=reach_at_destination_date_time+datetime.timedelta(hours=wait_hour)
#         ##### Include Honduras remolque restrictions here
#         #### Add the individual restrictions here
#         truck_reach_back_date_time_obtained=truck_reach_dest_date_time_obtained+\
#         datetime.timedelta(minutes=one_way_time_in_min)+datetime.timedelta(minutes=unloading_time_at_dest)+\
#         datetime.timedelta(minutes=break_time_min_dest)
#         duration_of_trip=(truck_reach_back_date_time_obtained - truck_start_date_time_obtained)/pd.Timedelta('1 hour')

# #     print("destination attention obtained",attention_end_obtained)
# #     print("can we send external truck",send_external_truck)
# #     print("can we send own truck",truck_sent)
# #     print("waiting time ",wait_hour)
# print(truck_reach_back_date_time_obtained,truck_start_date_time_obtained)

In [ ]:
two_way_time_in_min,unloading_time_at_dest,duration_of_trip_hr=check_destination_attention_v1(destiny_attention_df,distance_and_time,origin,dest1,attention_start_time_obtained,Max_reach_date_time_at_destiny_p,fecha_str,day_of_week)

In [ ]:
truck_start_date_time_obtained

In [ ]:
destiny_attention_df[destiny_attention_df['COD']=='DH03']

In [ ]:
if (restr_1!='No restriction') and (dest != restr_1):
    print('True')
else:
    print('False')

In [ ]:
distance_and_time[(distance_and_time['Origin_Code'] == origin) & (distance_and_time['Destiny_Code'] == destiny)]